# Header

In [1]:
#Loading Hyperspy Utilities and additional libraries
%matplotlib qt5
import numpy as np
import hyperspy
import hyperspy.api as hs
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import cv2
import sys
import os
from sympy import Symbol
from joblib import Parallel, delayed
import multiprocessing
hs.preferences.gui(toolkit='ipywidgets')

/home/max/Apps/anaconda3/envs/Hyperspy/lib/python3.6/site-packages/matplotlib/__init__.py:846: MatplotlibDeprecationWarning: 
The text.latex.unicode rcparam was deprecated in Matplotlib 2.2 and will be removed in 3.1.
  "2.2", name=key, obj_type="rcparam", addendum=addendum)
/home/max/Apps/anaconda3/envs/Hyperspy/lib/python3.6/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [2]:
#Loading libraries for watershed-analysis
import matplotlib as mpl
import matplotlib.image as mpimg

import mendeleev as mend
import sympy
import scipy
from scipy import ndimage as ndi
from skimage.filters import rank
from skimage.morphology import (
    watershed, disk, diamond, binary_erosion, binary_dilation)
from skimage.util import img_as_ubyte

In [19]:
sympy.init_printing()
x, t = sympy.symbols('x t', real=True)

mu = sympy.symbols('mu', real=True)
sigma, a, b, lamb, nu = sympy.symbols('sigma a b lambda nu', positive=True)

def area(dist):
    return sympy.simplify(sympy.integrate(dist, (x, -sympy.oo, sympy.oo)))
def mean(dist):
    return area(dist*x)
def EX2(dist):
    return area(dist*x**2)
def variance(dist):
    return sympy.simplify(EX2(dist) - mean(dist)**2)
def mgf(dist):
    return sympy.simplify(area(dist*sympy.exp(x*t)))
def latex(result):
    return "$" + sympy.latex(result) + "$\n" 
def summarize(dist):
    print("Distribution: " + latex(dist))
    print("Area: " + latex(area(dist)))
    print("Mean: " + latex(mean(dist)))
    print("Variance: " + latex(variance(dist)))
    print("MGF: " + latex(mgf(dist)))
summarise = summarize  # alias

In [22]:
gamma = sympy.Piecewise(
    (0, x < 0),
    (b**a / sympy.gamma(a) * x**(a-1) * sympy.exp(-x*b), True)
)
summarize(gamma)

Distribution: $\begin{cases} 0 & \text{for}\: x < 0 \\\frac{b^{a} x^{a - 1} e^{- b x}}{\Gamma\left(a\right)} & \text{otherwise} \end{cases}$

Area: $\int_{-\infty}^{\infty} \begin{cases} 0 & \text{for}\: x < 0 \\\frac{b^{a} x^{a - 1} e^{- b x}}{\Gamma\left(a\right)} & \text{otherwise} \end{cases}\, dx$

Mean: $\int_{-\infty}^{\infty} \begin{cases} 0 & \text{for}\: x < 0 \\\frac{\left(b x\right)^{a} e^{- b x}}{\Gamma\left(a\right)} & \text{otherwise} \end{cases}\, dx$

Variance: $\begin{cases} - \left(\int_{-\infty}^{\infty} 0\, dx\right)^{2} + \int_{-\infty}^{\infty} 0\, dx & \text{for}\: x < 0 \\- \left(\int_{-\infty}^{\infty} \frac{a b^{a} x^{a} e^{- b x}}{\Gamma\left(a + 1\right)}\, dx\right)^{2} + \int_{-\infty}^{\infty} \frac{a b^{a} x^{a + 1} e^{- b x}}{\Gamma\left(a + 1\right)}\, dx & \text{otherwise} \end{cases}$

MGF: $\frac{t^{- a} \left(\frac{b}{\operatorname{polar\_lift}{\left (- \frac{b}{t} + 1 \right )}}\right)^{a} e^{- i \pi a} \gamma\left(a, - \infty \operatorname{sign}

# Mapping automation _Plasmon-characteristics_ 

## Class _Plasmon feature Mapping_

In [55]:
# Class shall be used to map shear-transformation-zones for an image to visualize 
# shear-bands by considering energy-shifts of the plasmon-peaks aligned with the ZLP
class Plasmon_mapper(object): 

    # Initialize the class for STZ-mapping 
    def __init__(self, 
                 filename      = '', 
                 file          = None,
                 is_stack      = False, 
                 is_lazy       = False, 
                 binning       = True,
                 includes_fits = False,
                 normalize     = False
                ):
        """
        Plasmon_mapper: 
                    With careful analysis of an EELS-image the change of the spatial electron density
                    and additional thickness analysis, the MSER algorithm generates segments of the 
                    sheared regions and evaluates orientation of sheared regions, as the strain
                    impacts on the Plasmon resonance energy.
                    Different outputs are possible to analyze the parameter shifts of the plasmon 
                    signature in the specimen.
        ----------------------------------------------------------------------------------------------
        Initialization parameters:
                    filename: STRING      - specify filename and directory to load the EELS-image generated 
                                            (e.g. in GATAN)
                    file: Hyperspy-Signal - only use this when not using filename. Previously initia-
                                            lized Hyperspy Signals object. Read more on the Signals 
                                            objects http://hyperspy.org/hyperspy-doc/
                    is_stack: BOOLEAN     - loading multiple files by a wildcard "*"
                    
                    is_lazy: BOOLEAN      - to analyze big data (e.g. .dm4 - files) can be a problematic
                                            thing to due high memory requirements, therefore instead of
                                            using a numpy nd.array one can fall back to dask.dask_array
                                            resulting in an object which consists of multiple numpy arrays.
                                            This makes it possible to only work on parts of the datafile
                                            and therefore requiring only a small part of the memory require-
                                            ments.
                    binning: BOOLEAN      - operate on binned datasets. Due to caution set to FALSE as 
                                            default! Preferably, the datasets should only be binned using
                                            hyperspy to verify correct results.
        """
        self.Filename          = ''
        self.File              = None
        self.File_deconv       = None
        self.samf              = None
        
        self.Fit_Model         = None
        self.Chisq             = None
        self.red_Chisq         = None
        
        self.eels_axis         = None
        self.function_set      = None
        self.model_name        = None
        
        self.ZLP_Emax          = None
        self.ZLP_FWHM          = None
        self.ZLP_Int           = None
        
        self.FPP_Emax          = None
        self.FPP_FWHM          = None
        self.FPP_Int           = None
        self.SPP_Emax          = None
        self.SPP_FWHM          = None
        self.SPP_Int           = None
        
        self.Ep_q0             = None
        
        self.parameter_dict    = {}

        self.Elements          = None
        self.Concentrations    = None
        self.thickness_map     = None
        
        self.dir_list          = ['Parameter Maps', 
                                  'Thickness Evaluation', 
                                  'Density Analysis', 
                                  'Plasmon characteristics'
                                 ]
        
        self.ROI               = None
        self.Line              = None
        self.Line_Scan         = None
        
        self.elastic_threshold = None # arithmetic mean of the elastic threshold over the navigation space
        self.elastic_intensity = None
        
        self.load_data(filename, file, is_stack, is_lazy, binning)
        
    
    def print_file_information(self):
        """
        Printing standard metadata of file for beam and detector information as well as dataset information
        and the currently stored models of the file
        """
        print(self.File.metadata)
        
        print(self.File.axes_manager)
        
        print(self.File.models)
        
        
    def yes_or_no(self, question):
        """
        Function for yes-no user input. Returns BOOLEAN value (y = True | n = False)
        """
        reply = str(input(question+' (y/n): ')).lower().strip()
        
        if reply[0] == 'y':
            return True
        
        elif reply[0] == 'n':
            return False
        
        else:
            return self.yes_or_no("Please Enter")
      
    
    def model_gui(self, model):
        """
        Function to call the model attribute gui and await parameter adjustments until user is finished
        """
        model.gui()
        while True:
            finished = self.yes_or_no('When finished type (y) :')
            if (finished == True):
                break
        
    
    def split(self, txt, seps):
        """
        Splitting a string by specified seperators seps taking a tuple as input.
        """
        default_sep = seps[0]

        # we skip seps[0] because that's the default seperator
        for sep in seps[1:]:
            txt = txt.replace(sep, default_sep)
        return [i.strip() for i in txt.split(default_sep)]

        
    # Routine to load the data
    def load_data(self, 
                  filename, 
                  file, 
                  is_stack, 
                  is_lazy,
                  binning
                 ):
        """
        load_data: This routine specifies the order of dimensional alignment and
                   the zero-loss alignment as well as the usage of a ROI for specific
                   analysis while conserving the metadata of the the file. The last work 
                   step introducing the ROI will be dismissed in this routine in the next update. 
                   The parameters will be submitted from the __init__ routine of the STZ-Mapper class.
        ----------------------------------------------------------------------------------------------
        Initialization parameters:
                    filename: STRING - specify filename and directory to load the EELS-image generated 
                                       (e.g. in GATAN)
                    file: Hyperspy-Signal - only use this when not using filename. Previously initia-
                                            lized Hyperspy Signals object. Read more on the Signals 
                                            objects http://hyperspy.org/hyperspy-doc/
                    is_stack: BOOLEAN - loading multiple files by a wildcard "*"
                    
                    is_lazy: BOOLEAN - to analyze big data (e.g. .dm4 - files) can be a problematic
                                       thing to due high memory requirements, therefore instead of
                                       using a numpy nd.array one can fall back to dask.dask_array
                                       resulting in an object which consists of multiple numpy arrays.
                                       This makes it possible to only work on parts of the datafile
                                       and therefore requiring only a small part of the memory require-
                                       ments.
                    use_roi: BOOLEAN - depreciated! will be rewritten
        """
        if (filename  != ''):
            self.File  = hs.load(filename, stack = is_stack, lazy = is_lazy)

        elif (file    != None):
            self.File  = file

        self.Filename          = self.File.metadata.General.title
        
        try:
            self.eels_axis     = self.File.axes_manager['Energy loss']
        
        except:
            axes               = self.File.axes_manager.as_dictionary()
            for i in range(len(axes) - 1):
                axis = axes['axis-' + str(i)]
                if (axis['units'] == 'eV'):
                    self.eels_axis = self.File.axes_manager[i]

        including_fits = self.yes_or_no('Does the file include stored models from previous fitting?'
                                       )
        if (including_fits   == True):
            self.load_model()
        
        elif (including_fits == False):
            channels_before                     = self.File.axes_manager.signal_size
            self.File.data[self.File.data <= 0] = 1
            
            print('Setting up proper navigation space...')
            self.align_dataset()
            
            print('Aligning Zero-Loss Peak...')
            self.File.align_zero_loss_peak(subpixel = True)
            
            print('Aligning datastructure successful. Estimate poissonian noise...')
            self.File.estimate_poissonian_noise_variance()
            
            deconv     = self.yes_or_no('Do you want to continue with the deconvoluted spectrum using' +  '\n'
                                        'the Fourier-Log-Deconvolution method?' + '\n' +
                                        '[Less calculation effort in the fitting routine as only the' + '\n' 
                                        ' plasmon peak is fitted. Precision might decrease.' + '\n'
                                       )
            if (deconv == True):
                print('Calculating deconvoluted spectrum...')
                self.zlp_deconvolution()
            
            #norm           = self.yes_or_no('Shall the data be normalized?' + '\n' +
            #                                '[Normalization by integration over spectrum axis]' + '\n'
            #                               )
            #if (norm   == True):
            #    self.normalize(deconv)
            
            channels_after   = self.File.axes_manager.signal_size
            cropped_channels = channels_before - channels_after
            print('Loading process completed. Energy channels cropped by ZLP-Alignment: ' + 
                  str(cropped_channels) + '[' + str(cropped_channels/channels_before*100) + '%]' +
                  '\n' + 'Channels before: ' + str(channels_before) + '\n' +
                  'Channels after: ' + str(channels_after)
                 )

        
        self.elastic_threshold = np.mean(
            self.File.estimate_elastic_scattering_threshold().data
        )
        self.elastic_intensity = np.mean(
            self.File.estimate_elastic_scattering_intensity(self.elastic_threshold).data
        )
        self.File.metadata.Signal.binned = binning
        
    
    def align_dataset(self):
        """
        align_dataset: By dimension the dataset will be devided into different objects as follows:
                           EELS: 1D array specifying Counts/Intensity along the energy loss axis
                                 given by offset and gain as defined in the metadata
                           EELS-stack: 2D array specifying Counts/Intensity as above (see Object EELS)
                                       and 1 navigation axis
                           EELS-image: 3D array specifying Counts/Intensity as above (see Object EELS)
                                       and 2 navigation axis - therefore the name... :)
        ----------------------------------------------------------------------------------------------
        Initialization parameters:
                    None, all parameters should have been included in the metadata by GATAN or by
                    defining the meta information in hyperspy yourself.
        """
        #tmp = self.File.metadata.Signal.Noise_properties
        #variance = self.File.copy()
        print(self.eels_axis)
        self.File = self.File.as_signal1D(spectral_axis=self.eels_axis)#, out=self.File)
        #self.File.metadata.Signal.Noise_properties =tmp
        self.File.set_signal_type("EELS")
        
        for axis in self.File.axes_manager.navigation_axes:
            axis.offset = 0
            axis.scale  = abs(axis.scale)

    
    def zlp_deconvolution(self):
        """
        Uses Fourier-Log method for deconvolution and elastic scattering threshold to estimate ZLP.
        """
        zlp              = self.File.isig[:self.elastic_threshold]
        self.File_deconv = self.File.fourier_log_deconvolution(zlp)
        
    
    def normalize(self, 
                  deconv
                 ):
        """
        Normalize an EELS-spectrum / spectrum-image setting the maximum height of the zero-loss-peak to 1.
        Currently not needed!
        """
        #possibility for axis to be read automatically
        #axis=len(np.array(im).shape)-1
        if (deconv == False):
            poissonian_noise = self.File.metadata.Signal.get_item("Noise_properties.variance")
            integrated = self.File.integrate1D(self.File.axes_manager.signal_axes)
            self.File_norm = self.File / integrated
            self.File_norm.metadata.Signal.set_item("Noise_properties.variance", poissonian_noise/integrated)
        
        elif (deconv == True):
            poissonian_noise = self.File_deconv.metadata.Signal.get_item("Noise_properties.variance")
            integrated = self.File_deconv.integrate1D(self.File_deconv.axes_manager.signal_axes)
            self.File_norm = self.File_deconv / integrated
            self.File_norm.metadata.Signal.set_item("Noise_properties.variance", poissonian_noise/integrated)

        
    def normalize_parameter_maps(self):
        """
        Comments missing
        """
        #MISSING
        return 0


    def init_model_params(self, 
                          functions,
                          deconv,
                          normalized
                         ):
        """
        Comments missing
        """
        if (self.function_set != 'VolumePlasmonDrude' and self.function_set != 'Lorentzian'):
            print('Estimate function parameters...')
            
            if (normalized == True):
                
                if (deconv == False):
                    functions[0].estimate_parameters(self.File_norm, -10., 10.)
                    functions[1].estimate_parameters(self.File_norm, 10., 50.)
                    functions[2].estimate_parameters(self.File_norm, 25., 75.)

                elif (deconv == True):
                    functions[0].estimate_parameters(self.File_norm, 10., 50.)
                
            elif (normalized == False):
                
                if (deconv == False):
                    functions[0].estimate_parameters(self.File, -10., 10.)
                    functions[1].estimate_parameters(self.File, 10., 50.)
                    functions[2].estimate_parameters(self.File, 25., 75.)

                elif (deconv == True):
                    functions[0].estimate_parameters(self.File_deconv, 10., 50.)
                    
            else:
                print('The attributes normalized and deconv has to be BOOLEAN. Please set to "True" or "False".')
                return 0
    
    
    def init_func(self):
        """
        Initialize distribution type for ZLP, FPP, SPP. Three types are currently supported:
        func: {'VolumePlasmonDrude', 'Lorentzian', 'Gaussian', 'Voigt'}
        """
        Int  = self.elastic_intensity
        E0 = 0.  # in eV
        Ep = 15. # in eV
        
        try:
        
            if (self.function_set == 'VolumePlasmonDrude'):

                func_1 = hs.model.components1D.Voigt()
                func_1.area.value                = Int
                func_1.centre.value              = E0
                func_1.FWHM.value                = 0.1
                func_1.gamma.value               = 0.1
                func_1.name = 'Zero_Loss_Peak'

                func_2 = hs.model.components1D.VolumePlasmonDrude(intensity=Int*0.05,
                                                                  plasmon_energy=Ep,
                                                                  fwhm=0.1
                                                                 )
                func_2.name = 'First_Plasmon_Peak'

                func_3 = hs.model.components1D.VolumePlasmonDrude(intensity=Int*0.005,
                                                                  plasmon_energy=Ep*2,
                                                                  fwhm=0.1
                                                                 )
                func_3.name = 'Second_Plasmon_Peak'
                
            elif (self.function_set == 'Lorentzian'):

                func_1 = hs.model.components1D.Voigt()
                func_1.area.value                = Int
                func_1.centre.value              = E0
                func_1.FWHM.value                = 0.1
                func_1.gamma.value               = 0.1
                func_1.name = 'Zero_Loss_Peak'
                
                func_2 = hs.model.components1D.Lorentzian(A=Int*0.05, 
                                                          centre=Ep,
                                                          gamma=0.1
                                                         )
                func_2.name = 'First_Plasmon_Peak'

                func_3 = hs.model.components1D.Lorentzian(A=Int*0.005, 
                                                          centre=Ep*2,
                                                          gamma=0.1
                                                         )
                func_3.name = 'Second_Plasmon_Peak'
                
            elif (self.function_set == 'Gaussian'):

                func_1 = hs.model.components1D.Voigt()
                func_1.area.value                = Int
                func_1.centre.value              = E0
                func_1.FWHM.value                = 0.1
                func_1.gamma.value               = 0.1
                func_1.name = 'Zero_Loss_Peak'

                func_2 = hs.model.components1D.Gaussian(A=Int*0.05,
                                                        centre=Ep,
                                                        sigma=0.1
                                                       )
                func_2.name = 'First_Plasmon_Peak'

                func_3 = hs.model.components1D.Gaussian(A=Int*0.005,
                                                        centre=Ep*2,
                                                        sigma=0.1
                                                       )
                func_3.name = 'Second_Plasmon_Peak'

            elif (self.function_set == 'Voigt'):

                func_1 = hs.model.components1D.Voigt()
                func_1.area.value                = Int
                func_1.centre.value              = E0
                func_1.FWHM.value                = 0.1
                func_1.gamma.value               = 0.1
                func_1.name = 'Zero_Loss_Peak'
                
                func_2 = hs.model.components1D.Voigt()
                func_2.area.value   = Int*0.05
                func_2.centre.value = Ep
                func_2.FWHM.value   = 0.1
                func_2.gamma.value  = 0.1
                func_2.name = 'First_Plasmon_Peak'
                
                func_3 = hs.model.components1D.Voigt()
                func_2.area.value   = Int*0.005
                func_2.centre.value = Ep*2
                func_2.FWHM.value   = 0.1
                func_2.gamma.value  = 0.1
                func_3.name = 'Second_Plasmon_Peak'
                
            return func_1, func_2, func_3
            
        except:
            
            print('No distributions initialized. Please try again. For more information, see docstring.')
        

    
    # Function to fit EELS-Spectra with three gaussian
    # Another option is to use code in 2nd cell to use extra class
    def eels_fit_routine(self, 
                         function_set='VolumePlasmonDrude', 
                         fitter='leastsq', 
                         method='ls', 
                         samfire=False, 
                         multithreading=False, 
                         workers=4,
                         deconv=False,
                         normalized=False
                        ):         
        """
        Routine to initialize fit routine. Storing the calculated model in the loaded file.
        
        Initialize distribution model for ZLP, FPP, SPP. Three types are currently supported:
        function_Set: {'VolumePlasmonDrude', 'Lorentzian', 'Gaussian', 'Voigt'}
        """
        # Initialize serial fitting routine
        self.function_set = function_set
        self.model_name = function_set + str('_') + method
        
        if (samfire == False):

            self.Fit_Model = self.fit_eels(fitter, 
                                           method,
                                           deconv,
                                           normalized
                                          )

        # Initialize SAMFire (parallel) fitting routine
        else:

            self.Fit_Model = self.fit_eels_SAMF(fitter, 
                                                method, 
                                                multithreading, 
                                                workers,
                                                deconv,
                                                normalized
                                               )
        
        print('Model will be stored in file...')        
        self.Fit_Model.store(name = self.model_name)
        print('Stored models in file:')
        
        if (normalized == True):
            print(self.File_norm.models)
            
        else:
            if (deconv == False):
                print(self.File.models)

            elif (deconv == True):
                print(self.File_deconv.models)
            
        self.generate_param_maps()
        
        
    def init_model(self, 
                   deconv, 
                   normalized,
                   mean = False
                  ):
        """
        Comments missing
        """
        if (mean == True):
            
            if (deconv == True):
                mean                = self.File_deconv.mean(),create_model(auto_background=False)
                
            elif (deconv == False):
                mean                = self.File.mean().create_model(auto_background=False)
                
            func1, func2, func3 = self.init_func()

            mean.set_signal_range(-self.elastic_threshold, 100.) # in eV
            mean.extend([func1, func2, func3])

            self.set_second_plasmonenergy(mean)
            
            return mean
        
        elif (mean == False):
            
            if (deconv == True):
                model               = self.File_deconv.create_model(auto_background=False)
            
            elif (deconv == False):
                model               = self.File.create_model(auto_background=False)
                
            func1, func2, func3 = self.init_func()

            model.set_signal_range(-self.elastic_threshold, 100.) # in eV
            model.extend([func1, func2, func3])

            gui = self.yes_or_no('Do you want to use the gui to manually set the starting parameters?')
            if (gui == True):
                self.model_gui(model)

            else:
                self.init_model_params([func1, func2, func3], deconv, normalized)

            self.set_second_plasmonenergy(model)
            
            return model
    
    
    def set_second_plasmonenergy(self, model):
        """
        Comments missing
        """
        if (self.function_set != 'VolumePlasmonDrude'):
            model.components.Second_Plasmon_Peak.centre.twin_function_expr         = '2*x'
            model.components.Second_Plasmon_Peak.centre.inverse_twin_function_expr = 'x/2'
            model.components.Second_Plasmon_Peak.centre.twin                       = model.components.First_Plasmon_Peak.centre
            #model.components.Second_Plasmon_Peak.centre.free                       = False

        else:
            model.components.Second_Plasmon_Peak.plasmon_energy.twin_function_expr         = '2*x'
            model.components.Second_Plasmon_Peak.plasmon_energy.inverse_twin_function_expr = 'x/2'
            model.components.Second_Plasmon_Peak.plasmon_energy.twin                       = model.components.First_Plasmon_Peak.plasmon_energy
            #model.components.Second_Plasmon_Peak.plasmon_energy.free                       = False
    
    
    def fit_zlp_only(self, 
                     model, 
                     fitter, 
                     method,
                     bounded
                    ):
        """
        Comments missing
        """
        mean = self.init_model(False, False, mean = True)
        
        self.set_bounds(mean, bounded)
        
        mean.components.Zero_Loss_Peak.active      = True
        mean.components.First_Plasmon_Peak.active  = False
        mean.components.Second_Plasmon_Peak.active = False

        mean.set_signal_range(-self.elastic_threshold, 
                              self.elastic_threshold
                             )
        
        mean.fit(fitter=fitter, 
                 method=method,
                 bounded=bounded
                )
        
        self.set_model_params(model, 
                              mean, 
                              func='Zero_Loss_Peak'
                             )
        
        
    def fit_pp_only(self,
                    upper_bound,
                    model, 
                    fitter, 
                    method,
                    bounded
                   ):
        """
        Comments missing
        """
        mean = self.init_model(False, False, mean = True)
        
        self.set_bounds(mean, bounded)
        
        mean.components.Zero_Loss_Peak.active      = False
        mean.components.First_Plasmon_Peak.active  = True
        mean.components.Second_Plasmon_Peak.active = True

        self.set_second_plasmonenergy(mean)
        
        mean.set_signal_range(self.elastic_threshold, #starting range with inflexion point of ZLP
                              upper_bound #eV
                             )

        mean.fit(fitter=fitter, 
                 method=method,
                 bounded=bounded
                )
        
        if (self.function_set == 'VolumePlasmonDrude'):
            energy_fpp = mean.components.First_Plasmon_Peak.plasmon_energy.as_signal(field='values').data

            tmp = energy_fpp[energy_fpp > self.elastic_threshold]
            plasmonenergy = np.mean(tmp[tmp < 2 * np.mean(tmp)])
            #mean.components.Second_Plasmon_Peak.plasmon_energy.free = False


        else:
            energy_fpp = mean.components.First_Plasmon_Peak.centre.as_signal(field='values').data
            #mean.components.Second_Plasmon_Peak.centre.free = False

            tmp = energy_fpp[energy_fpp > self.elastic_threshold]
            plasmonenergy = np.mean(tmp[tmp < 2 * np.mean(tmp)])
            
        mean.set_signal_range(self.elastic_threshold, #see above
                              3 * plasmonenergy #using better estimate of plasmon energy range to increase accuracy 
                             )

        mean.fit(fitter=fitter, 
                 method=method,
                 bounded=bounded
                )
        
        self.set_model_params(model, 
                              mean, 
                              func='First_Plasmon_Peak'
                             )
        
        self.set_model_params(model, 
                              mean, 
                              func='Second_Plasmon_Peak'
                             )
        
        return plasmonenergy
        
    
    def fit_fpp_only(self, 
                     upper_bound,
                     model, 
                     fitter, 
                     method,
                     bounded
                    ):
        """
        Comments missing
        """
        mean = self.init_model(False, False, mean = True)
        
        self.set_bounds(mean, bounded)
        
        mean.components.Zero_Loss_Peak.active      = False
        mean.components.First_Plasmon_Peak.active  = True
        mean.components.Second_Plasmon_Peak.active = False

        mean.set_signal_range(self.elastic_threshold, #see above
                              upper_bound #eV
                             )
        
        mean.fit(fitter=fitter, 
                 method=method,
                 bounded=bounded
                )
        
        if (self.function_set == 'VolumePlasmonDrude'):
            energy_fpp = mean.components.First_Plasmon_Peak.plasmon_energy.as_signal(field='values').data
            tmp = energy_fpp[energy_fpp > self.elastic_threshold]
            plasmonenergy = np.mean(tmp[tmp < 2 * np.mean(tmp)])


        else:
            energy_fpp = mean.components.First_Plasmon_Peak.centre.as_signal(field='values').data
            tmp = energy_fpp[energy_fpp > self.elastic_threshold]
            plasmonenergy = np.mean(tmp[tmp < 2 * np.mean(tmp)])
            
        
        mean.set_signal_range(self.elastic_threshold, #see above
                              2 * plasmonenergy #using better estimate of plasmon energy range to increase accuracy 
                             )

        mean.fit(fitter=fitter, 
                 method=method,
                 bounded=bounded
                )
        
        self.set_model_params(model, 
                              mean, 
                              func='First_Plasmon_Peak'
                             )
        
        return plasmonenergy
        
        
    def set_bounds(self, model, bounded):
        """
        Comments missing
        """
        if (self.function_set == 'Voigt'):
            
            if (bounded):
                model.components.Zero_Loss_Peak.area.bmin   = (self.elastic_intensity 
                                                               - self.elastic_intensity / 2
                                                              )
                model.components.Zero_Loss_Peak.area.bmax   = (self.elastic_intensity 
                                                               + self.elastic_intensity / 2
                                                              )
                model.components.Zero_Loss_Peak.centre.bmin = -self.elastic_threshold
                model.components.Zero_Loss_Peak.centre.bmax = self.elastic_threshold
                model.components.Zero_Loss_Peak.FWHM.bmin   = 0.
                model.components.Zero_Loss_Peak.FWHM.bmax   = 2 * self.elastic_threshold
                model.components.Zero_Loss_Peak.gamma.bmin  = 0.
                model.components.Zero_Loss_Peak.gamma.bmax  = 2 * self.elastic_threshold


                model.components.First_Plasmon_Peak.area.bmin   = 0.
                model.components.First_Plasmon_Peak.area.bmax   = (self.elastic_intensity 
                                                                   + self.elastic_intensity / 2
                                                                  )
                model.components.First_Plasmon_Peak.centre.bmin = self.elastic_threshold
                model.components.First_Plasmon_Peak.centre.bmax = 50.
                model.components.First_Plasmon_Peak.FWHM.bmin   = 0.
                model.components.First_Plasmon_Peak.FWHM.bmax   = 50.
                model.components.First_Plasmon_Peak.gamma.bmin  = 0.
                model.components.First_Plasmon_Peak.gamma.bmax  = 50.


                model.components.Second_Plasmon_Peak.area.bmin   = 0.
                model.components.Second_Plasmon_Peak.area.bmax   = (self.elastic_intensity 
                                                                    + self.elastic_intensity / 2
                                                                   )
                model.components.Second_Plasmon_Peak.centre.bmin = self.elastic_threshold
                model.components.Second_Plasmon_Peak.centre.bmax = 100.
                model.components.Second_Plasmon_Peak.FWHM.bmin   = 0.
                model.components.Second_Plasmon_Peak.FWHM.bmax   = 100.
                model.components.Second_Plasmon_Peak.gamma.bmin  = 0.
                model.components.Second_Plasmon_Peak.gamma.bmax  = 100.
                
            
            else:
                model.components.Zero_Loss_Peak.area.ext_force_positive        = True
                model.components.Zero_Loss_Peak.centre.ext_force_positive      = False
                model.components.Zero_Loss_Peak.FWHM.ext_force_positive        = True
                model.components.Zero_Loss_Peak.gamma.ext_force_positive       = True
                model.components.First_Plasmon_Peak.area.ext_force_positive    = True
                model.components.First_Plasmon_Peak.centre.ext_force_positive  = True
                model.components.First_Plasmon_Peak.FWHM.ext_force_positive    = True
                model.components.First_Plasmon_Peak.gamma.ext_force_positive   = True
                model.components.Second_Plasmon_Peak.area.ext_force_positive   = True
                model.components.Second_Plasmon_Peak.centre.ext_force_positive = True
                model.components.Second_Plasmon_Peak.FWHM.ext_force_positive   = True
                model.components.Second_Plasmon_Peak.gamma.ext_force_positive  = True
        
        
        elif (self.function_set == 'Lorentzian'):
            
            if (bounded):
                model.components.Zero_Loss_Peak.area.bmin   = (self.elastic_intensity 
                                                               - self.elastic_intensity / 2
                                                              )
                model.components.Zero_Loss_Peak.area.bmax   = (self.elastic_intensity 
                                                               + self.elastic_intensity / 2
                                                              )
                model.components.Zero_Loss_Peak.centre.bmin = -self.elastic_threshold
                model.components.Zero_Loss_Peak.centre.bmax = self.elastic_threshold
                model.components.Zero_Loss_Peak.FWHM.bmin   = 0.
                model.components.Zero_Loss_Peak.FWHM.bmax   = 2 * self.elastic_threshold
                model.components.Zero_Loss_Peak.gamma.bmin  = 0.
                model.components.Zero_Loss_Peak.gamma.bmax  = 2 * self.elastic_threshold


                model.components.First_Plasmon_Peak.A.bmin      = 0.
                model.components.First_Plasmon_Peak.A.bmax      = (self.elastic_intensity 
                                                                   + self.elastic_intensity / 2
                                                                  )
                model.components.First_Plasmon_Peak.centre.bmin = self.elastic_threshold
                model.components.First_Plasmon_Peak.centre.bmax = 50.
                model.components.First_Plasmon_Peak.gamma.bmin  = 0.
                model.components.First_Plasmon_Peak.gamma.bmax  = 50.


                model.components.Second_Plasmon_Peak.A.bmin      = 0.
                model.components.Second_Plasmon_Peak.A.bmax      = (self.elastic_intensity 
                                                                    + self.elastic_intensity / 2
                                                                   )
                model.components.Second_Plasmon_Peak.centre.bmin = self.elastic_threshold
                model.components.Second_Plasmon_Peak.centre.bmax = 100.
                model.components.Second_Plasmon_Peak.gamma.bmin  = 0.
                model.components.Second_Plasmon_Peak.gamma.bmax  = 100.

            
            else:
                model.components.Zero_Loss_Peak.area.ext_force_positive        = True
                model.components.Zero_Loss_Peak.centre.ext_force_positive      = False
                model.components.Zero_Loss_Peak.FWHM.ext_force_positive        = True
                model.components.Zero_Loss_Peak.gamma.ext_force_positive       = True
                model.components.First_Plasmon_Peak.A.ext_force_positive       = True
                model.components.First_Plasmon_Peak.centre.ext_force_positive  = True
                model.components.First_Plasmon_Peak.gamma.ext_force_positive   = True
                model.components.Second_Plasmon_Peak.A.ext_force_positive      = True
                model.components.Second_Plasmon_Peak.centre.ext_force_positive = True
                model.components.Second_Plasmon_Peak.gamma.ext_force_positive  = True
                
                
        elif (self.function_set == 'Gaussian'):
            
            if (bounded):
                model.components.Zero_Loss_Peak.area.bmin   = (self.elastic_intensity 
                                                               - self.elastic_intensity / 2
                                                              )
                model.components.Zero_Loss_Peak.area.bmax   = (self.elastic_intensity 
                                                               + self.elastic_intensity / 2
                                                              )
                model.components.Zero_Loss_Peak.centre.bmin = -self.elastic_threshold
                model.components.Zero_Loss_Peak.centre.bmax = self.elastic_threshold
                model.components.Zero_Loss_Peak.FWHM.bmin   = 0.
                model.components.Zero_Loss_Peak.FWHM.bmax   = 2 * self.elastic_threshold
                model.components.Zero_Loss_Peak.gamma.bmin  = 0.
                model.components.Zero_Loss_Peak.gamma.bmax  = 2 * self.elastic_threshold


                model.components.First_Plasmon_Peak.A.bmin      = 0.
                model.components.First_Plasmon_Peak.A.bmax      = (self.elastic_intensity 
                                                                   + self.elastic_intensity / 2
                                                                  )
                model.components.First_Plasmon_Peak.centre.bmin = self.elastic_threshold
                model.components.First_Plasmon_Peak.centre.bmax = 50.
                model.components.First_Plasmon_Peak.sigma.bmin  = 0.
                model.components.First_Plasmon_Peak.sigma.bmax  = 50.


                model.components.Second_Plasmon_Peak.A.bmin      = 0.
                model.components.Second_Plasmon_Peak.A.bmax      = (self.elastic_intensity 
                                                                    + self.elastic_intensity / 2
                                                                   )
                model.components.Second_Plasmon_Peak.centre.bmin = self.elastic_threshold
                model.components.Second_Plasmon_Peak.centre.bmax = 100.
                model.components.Second_Plasmon_Peak.sigma.bmin  = 0.
                model.components.Second_Plasmon_Peak.sigma.bmax  = 100.
            

            else:
                model.components.Zero_Loss_Peak.area.ext_force_positive        = True
                model.components.Zero_Loss_Peak.centre.ext_force_positive      = False
                model.components.Zero_Loss_Peak.FWHM.ext_force_positive        = True
                model.components.Zero_Loss_Peak.gamma.ext_force_positive       = True
                model.components.First_Plasmon_Peak.A.ext_force_positive       = True
                model.components.First_Plasmon_Peak.centre.ext_force_positive  = True
                model.components.First_Plasmon_Peak.sigma.ext_force_positive   = True
                model.components.Second_Plasmon_Peak.A.ext_force_positive      = True
                model.components.Second_Plasmon_Peak.centre.ext_force_positive = True
                model.components.Second_Plasmon_Peak.sigma.ext_force_positive  = True
                
            
        elif (self.function_set == 'VolumePlasmonDrude'):
            
            if (bounded):
                model.components.Zero_Loss_Peak.area.bmin   = (self.elastic_intensity 
                                                               - self.elastic_intensity / 2
                                                              )
                model.components.Zero_Loss_Peak.area.bmax   = (self.elastic_intensity 
                                                               + self.elastic_intensity / 2
                                                              )
                model.components.Zero_Loss_Peak.centre.bmin = -self.elastic_threshold
                model.components.Zero_Loss_Peak.centre.bmax = self.elastic_threshold
                model.components.Zero_Loss_Peak.FWHM.bmin   = 0.
                model.components.Zero_Loss_Peak.FWHM.bmax   = 2 * self.elastic_threshold
                model.components.Zero_Loss_Peak.gamma.bmin  = 0.
                model.components.Zero_Loss_Peak.gamma.bmax  = 2 * self.elastic_threshold


                model.components.First_Plasmon_Peak.intensity.bmin      = 0.
                model.components.First_Plasmon_Peak.intensity.bmax      = (self.elastic_intensity 
                                                                   + self.elastic_intensity / 2
                                                                  )
                model.components.First_Plasmon_Peak.plasmon_energy.bmin = self.elastic_threshold
                model.components.First_Plasmon_Peak.plasmon_energy.bmax = 50.
                model.components.First_Plasmon_Peak.fwhm.bmin  = 0.
                model.components.First_Plasmon_Peak.fwhm.bmax  = 50.


                model.components.Second_Plasmon_Peak.intensity.bmin      = 0.
                model.components.Second_Plasmon_Peak.intensity.bmax      = (self.elastic_intensity 
                                                                    + self.elastic_intensity / 2
                                                                   )
                model.components.Second_Plasmon_Peak.plasmon_energy.bmin = self.elastic_threshold
                model.components.Second_Plasmon_Peak.plasmon_energy.bmax = 100.
                model.components.Second_Plasmon_Peak.fwhm.bmin  = 0.
                model.components.Second_Plasmon_Peak.fwhm.bmax  = 100.


            else:
                model.components.Zero_Loss_Peak.area.ext_force_positive                = True
                model.components.Zero_Loss_Peak.centre.ext_force_positive              = False
                model.components.Zero_Loss_Peak.FWHM.ext_force_positive                = True
                model.components.Zero_Loss_Peak.gamma.ext_force_positive               = True
                model.components.First_Plasmon_Peak.intensity.ext_force_positive       = True
                model.components.First_Plasmon_Peak.plasmon_energy.ext_force_positive  = True
                model.components.First_Plasmon_Peak.fwhm.ext_force_positive            = True
                model.components.Second_Plasmon_Peak.intensity.ext_force_positive      = True
                model.components.Second_Plasmon_Peak.plasmon_energy.ext_force_positive = True
                model.components.Second_Plasmon_Peak.fwhm.ext_force_positive           = True
                
    
    def set_model_params(self, 
                         model, 
                         mean, 
                         func='Zero_Loss_Peak'
                        ):
        
        func_dict = {'Zero_Loss_Peak'      : mean.components.Zero_Loss_Peak,
                     'First_Plasmon_Peak'  : mean.components.First_Plasmon_Peak,
                     'Second_Plasmon_Peak' : mean.components.Second_Plasmon_Peak
                    }
        
        if (self.function_set == 'Voigt' or func == 'Zero_Loss_Peak'):
            model.set_parameters_value('area',
                                       func_dict[func].area.value,
                                       component_list=[func]
                                      )

            model.set_parameters_value('gamma',
                                       func_dict[func].gamma.value,
                                       component_list=[func]
                                      )

            model.set_parameters_value('FWHM',
                                       func_dict[func].FWHM.value,
                                       component_list=[func]
                                      )
            
            if (func != 'Second_Plasmon_Peak'):
                model.set_parameters_value('centre',
                                           func_dict[func].centre.value,
                                           component_list=[func]
                                          )
        
        if (self.function_set == 'Lorentzian' and func != 'Zero_Loss_Peak'):
            model.set_parameters_value('A',
                                       func_dict[func].A.value,
                                       component_list=[func]
                                      )

            model.set_parameters_value('gamma',
                                       func_dict[func].gamma.value,
                                       component_list=[func]
                                      )

            if (func != 'Second_Plasmon_Peak'):
                model.set_parameters_value('centre',
                                           func_dict[func].centre.value,
                                           component_list=[func]
                                          )
            
        if (self.function_set == 'Gaussian' and func != 'Zero_Loss_Peak'):
            model.set_parameters_value('A',
                                       func_dict[func].area.value,
                                       component_list=[func]
                                      )

            model.set_parameters_value('sigma',
                                       func_dict[func].gamma.value,
                                       component_list=[func]
                                      )

            if (func != 'Second_Plasmon_Peak'):
                model.set_parameters_value('centre',
                                           func_dict[func].centre.value,
                                           component_list=[func]
                                          )
            
        if (self.function_set == 'VolumePlasmonDrude' and func != 'Zero_Loss_Peak'):
            model.set_parameters_value('intensity',
                                       func_dict[func].intensity.value,
                                       component_list=[func]
                                      )

            model.set_parameters_value('fwhm',
                                       func_dict[func].fwhm.value,
                                       component_list=[func]
                                      )

            if (func != 'Second_Plasmon_Peak'):
                model.set_parameters_value('plasmon_energy',
                                           func_dict[func].plasmon_energy.value,
                                           component_list=[func]
                                          )
    
    
    def fit_eels(self, 
                 fitter, 
                 method,
                 deconv,
                 normalized
                ):
        """
        Comments missing
        """
        model = self.init_model(deconv, 
                                normalized
                               )
        
        if (method == 'ls'):
            bounded = True
        
        else:
            bounded = False
            
        self.set_bounds(model, bounded)
            
        upper_bound = 100. #eV
        
        if (deconv == False):
            self.set_second_plasmonenergy(model)
            
            model.print_current_values()
            self.fit_zlp_only(model, 
                              fitter, 
                              method,
                              bounded
                             )
            
            plasmonenergy = self.fit_pp_only(upper_bound,
                                             model, 
                                             fitter, 
                                             method,
                                             bounded
                                            )
            model.print_current_values()

            model.components.Zero_Loss_Peak.active      = True
            model.components.First_Plasmon_Peak.active  = True
            model.components.Second_Plasmon_Peak.active = True
            
            model.set_signal_range(-self.elastic_threshold, 
                                   3 * plasmonenergy
                                  )
            
            model.fit(fitter=fitter,
                      method=method,
                      bounded=bounded
                     )
            
            red_chisq = model.red_chisq.data
            red_chisq = np.mean(red_chisq[np.invert(np.isnan(red_chisq))])
            
            self.File.estimate_poissonian_noise_variance(gain_factor=red_chisq)
            
            
            model.multifit(fitter=fitter, 
                           method=method,
                           bounded=bounded
                          )
            model.print_current_values()
        
        elif (deconv == True):
            model.components.Zero_Loss_Peak.active      = False
            model.components.First_Plasmon_Peak.active  = True
            model.components.Second_Plasmon_Peak.active = False
            
            plasmonenergy = self.fit_fpp_only(upper_bound,
                                              model, 
                                              fitter, 
                                              method
                                             )
            
            model.set_signal_range(self.elastic_threshold, 
                                   2 * plasmonenergy
                                  )
            
            model.fit(fitter=fitter,
                      method=method,
                      bounded=bounded
                     )
            
            red_chisq = model.red_chisq.data
            red_chisq = np.mean(red_chisq[np.invert(np.isnan(red_chisq))])
            
            self.File.estimate_poissonian_noise_variance(gain_factor=red_chisq)
            
            model.multifit(fitter=fitter, 
                           method=method,
                           bounded=bounded
                          )
        
        return model
    
    
    def fit_eels_SAMF(self, 
                      fitter, 
                      method, 
                      multithreading, 
                      workers,
                      deconv,
                      normalized
                     ):
        """
        Comments missing
        """
        model = self.init_model(deconv, 
                                normalized
                               )
        
        if (method == 'ml'):
            bounded = False
        
        else:
            bounded = True
        
        self.set_bounds(model, bounded)
            
        upper_bound = 100. #eV
        
        if (deconv == False):
            self.set_second_plasmonenergy(model)
            
            self.fit_zlp_only(model, 
                              fitter, 
                              method,
                              bounded
                             )
            
            self.fit_pp_only(upper_bound,
                             model, 
                             fitter, 
                             method,
                             bounded
                            )
            
            if (self.function_set == 'VolumePlasmonDrude'):
                plasmonenergy = np.mean(model.components.First_Plasmon_Peak.plasmon_energy.as_signal(field='values').data)
                
            
            else:
                plasmonenergy = np.mean(model.components.First_Plasmon_Peak.centre.as_signal(field='values').data)
            
            model.components.Zero_Loss_Peak.active      = True
            model.components.First_Plasmon_Peak.active  = True
            model.components.Second_Plasmon_Peak.active = True
            
            model.set_signal_range(-self.elastic_threshold, 
                                   3 * plasmonenergy
                                  )
            
            model.fit(fitter=fitter, 
                      method=method,
                      bounded=bounded
                     )
            
            red_chisq = model.red_chisq.data
            red_chisq = np.mean(red_chisq[np.invert(np.isnan(red_chisq))])
            
            self.File.estimate_poissonian_noise_variance(gain_factor=red_chisq)
            
            if (model.axes_manager.navigation_dimension == 1):
            
                for i in range(10):
                    step_x = int(self.File.axes_manager.navigation_shape[0]/10)

                    model.axes_manager.indices = (int(step_x/2+i*step_x),)

                    model.fit(fitter=fitter, 
                              method=method,
                              bounded=bounded
                             )
                
        
            elif (model.axes_manager.navigation_dimension == 2):

                for i in range(10):
                    for j in range(10):
                        step_x = int(self.File.axes_manager.navigation_shape[0]/10)
                        step_y = int(self.File.axes_manager.navigation_shape[1]/10)

                        model.axes_manager.indices = (int(step_x/2 + i*step_x), int(step_y/2 + j*step_y))

                        model.fit(fitter=fitter, 
                                  method=method,
                                  bounded=bounded
                                 )
                        
            
            red_chisq = model.red_chisq.data
            mean_rchisq = np.mean(red_chisq[np.invert(np.isnan(red_chisq))])
            
            samf = model.create_samfire(workers=workers, 
                                        ipyparallel=multithreading, 
                                        setup=True
                                       )

            samf.metadata.goodness_test.tolerance = mean_rchisq * 1.5
            samf.remove(1)
            samf.refresh_database()

            samf.start(fitter=fitter, 
                       method=method,
                       bounded=bounded
                      )

            plt.close()
            
        
        elif (deconv == True):
            model.components.Zero_Loss_Peak.active      = False
            model.components.First_Plasmon_Peak.active  = True
            model.components.Second_Plasmon_Peak.active = False
            
            plasmonenergy = self.fit_fpp_only(upper_bound,
                                              model, 
                                              fitter,
                                              method,
                                              bounded
                                             )
            
            model.set_signal_range(self.elastic_threshold, 
                                   2 * plasmonenergy
                                  )
            
            model.fit(fitter=fitter, 
                      method=method,
                      bounded=bounded
                     )
            
            red_chisq = model.red_chisq.data
            red_chisq = np.mean(red_chisq[np.invert(np.isnan(red_chisq))])
            
            self.File.estimate_poissonian_noise_variance(gain_factor=red_chisq)
            
            if (model.axes_manager.navigation_dimension == 1):
            
                for i in range(10):
                    step_x = int(self.File.axes_manager.navigation_shape[0]/10)

                    model.axes_manager.indices = (int(step_x/2+i*step_x),)

                    model.fit(fitter=fitter, 
                              method=method,
                              bounded=bounded
                             )
                
        
            elif (model.axes_manager.navigation_dimension == 2):

                for i in range(10):
                    for j in range(10):
                        step_x = int(self.File.axes_manager.navigation_shape[0]/10)
                        step_y = int(self.File.axes_manager.navigation_shape[1]/10)

                        model.axes_manager.indices = (int(step_x/2 + i*step_x), int(step_y/2 + j*step_y))

                        model.fit(fitter=fitter, 
                                  method=method,
                                  bounded=bounded
                                 )
                        
            
            red_chisq = model.red_chisq.data
            mean_rchisq = np.mean(red_chisq[np.invert(np.isnan(red_chisq))])
            
            samf = model.create_samfire(workers=workers, 
                                        ipyparallel=multithreading, 
                                        setup=True
                                       )

            samf.metadata.goodness_test.tolerance = mean_rchisq * 1.5
            samf.remove(1)
            samf.refresh_database()

            samf.start(fitter=fitter, 
                       method=method,
                       bounded=bounded
                      )
            
        return model
    
    
    def save_models_to_file(self, 
                            filename
                           ):
        """
        Comments missing
        """
        self.File.save(filename)

    
    def load_model(self):
        """
        Comments missing
        """
        models_dict = { '1' : 'VolumePlasmonDrude_ls',
                        '2' : 'VolumePlasmonDrude_ml',
                        '3' : 'Lorentzian_ls',
                        '4' : 'Lorentzian_ml',
                        '5' : 'Gaussian_ls',
                        '6' : 'Gaussian_ml',
                        '7' : 'Voigt_ls',
                        '8' : 'Voigt_ml'
                      }
        print('Available models: \n' + str(self.File.models))
        print('Model names: \n' + str(models_dict))

        mname = input("Which model should be loaded? ")
        
        try:
            if (mname in models_dict):
                print('Loading parameter maps for: ' + str(models_dict[mname]))
                self.model_name = models_dict[mname]
            
                self.restore_model()
                self.function_set = self.split(models_dict[mname], ('_'))[0]
                self.Chisq        = self.Fit_Model.chisq
                self.red_Chisq    = self.Fit_Model.red_chisq
                
            else:
                print('Loading parameter maps for: ' + str(mname))
                self.model_name = mname

                self.restore_model()
                self.function_set = self.split(models_dict[mname], ('_'))[0]
                self.Chisq        = self.Fit_Model.chisq
                self.red_Chisq    = self.Fit_Model.red_chisq
                
            self.generate_param_maps()
                
        except:
            print('Model not found in files models-attribute. No model was loaded, please try again.')
                    
        
    def restore_model(self):
        """
        Comments missing
        """
        self.Fit_Model  = self.File.models.restore(self.model_name)
            
        
    def generate_param_maps(self):
        """
        Comments missing
        """
        if (self.function_set == 'VolumePlasmonDrude'):
        
            ZLP_L_fwhm      = self.Fit_Model.components.Zero_Loss_Peak.gamma.as_signal(field = 'values')
            ZLP_G_fwhm      = self.Fit_Model.components.Zero_Loss_Peak.FWHM.as_signal(field = 'values')
            self.ZLP_FWHM   = (ZLP_L_fwhm*0.5346 + 
                               np.sqrt(ZLP_G_fwhm**2 + ZLP_L_fwhm**2*0.2166)
                              )
            self.ZLP_Emax   = self.Fit_Model.components.Zero_Loss_Peak.centre.as_signal(field = 'values')
            self.ZLP_Int    = self.Fit_Model.components.Zero_Loss_Peak.area.as_signal(field = 'values')

            self.FPP_FWHM   = self.Fit_Model.components.First_Plasmon_Peak.fwhm.as_signal(field = 'values')
            self.FPP_Emax   = self.Fit_Model.components.First_Plasmon_Peak.plasmon_energy.as_signal(field = 'values')
            self.FPP_Int    = self.Fit_Model.components.First_Plasmon_Peak.intensity.as_signal(field = 'values')

            self.SPP_FWHM   = self.Fit_Model.components.Second_Plasmon_Peak.fwhm.as_signal(field = 'values')
            self.SPP_Emax   = self.Fit_Model.components.Second_Plasmon_Peak.plasmon_energy.as_signal(field = 'values')
            self.SPP_Int    = self.Fit_Model.components.Second_Plasmon_Peak.intensity.as_signal(field = 'values')

            self.Ep_q0      = ( self.FPP_Emax**2 - (self.FPP_FWHM / 2)**2 )**0.5
            
        elif (self.function_set == 'Lorentzian'):
        
            ZLP_L_fwhm      = self.Fit_Model.components.Zero_Loss_Peak.gamma.as_signal(field = 'values')
            ZLP_G_fwhm      = self.Fit_Model.components.Zero_Loss_Peak.FWHM.as_signal(field = 'values')
            self.ZLP_FWHM   = (ZLP_L_fwhm*0.5346 + 
                               np.sqrt(ZLP_G_fwhm**2 + ZLP_L_fwhm**2*0.2166)
                              )
            self.ZLP_Emax   = self.Fit_Model.components.Zero_Loss_Peak.centre.as_signal(field = 'values')
            self.ZLP_Int    = self.Fit_Model.components.Zero_Loss_Peak.area.as_signal(field = 'values')

            self.FPP_FWHM   = self.Fit_Model.components.First_Plasmon_Peak.gamma.as_signal(field = 'values')
            self.FPP_Emax   = self.Fit_Model.components.First_Plasmon_Peak.centre.as_signal(field = 'values')
            self.FPP_Int    = self.Fit_Model.components.First_Plasmon_Peak.A.as_signal(field = 'values')

            self.SPP_FWHM   = self.Fit_Model.components.Second_Plasmon_Peak.gamma.as_signal(field = 'values')
            self.SPP_Emax   = self.Fit_Model.components.Second_Plasmon_Peak.centre.as_signal(field = 'values')
            self.SPP_Int    = self.Fit_Model.components.Second_Plasmon_Peak.A.as_signal(field = 'values')

            self.Ep_q0      = ( self.FPP_Emax**2 - (self.FPP_FWHM / 2)**2 )**0.5
        
        elif (self.function_set == 'Gaussian'):
        
            ZLP_L_fwhm      = self.Fit_Model.components.Zero_Loss_Peak.gamma.as_signal(field = 'values')
            ZLP_G_fwhm      = self.Fit_Model.components.Zero_Loss_Peak.FWHM.as_signal(field = 'values')
            self.ZLP_FWHM   = (ZLP_L_fwhm*0.5346 + 
                               np.sqrt(ZLP_G_fwhm**2 + ZLP_L_fwhm**2*0.2166)
                              )
            self.ZLP_Emax   = self.Fit_Model.components.Zero_Loss_Peak.centre.as_signal(field = 'values')
            self.ZLP_Int    = self.Fit_Model.components.Zero_Loss_Peak.area.as_signal(field = 'values')

            self.FPP_FWHM   = 2*np.sqrt(np.log(2))*self.Fit_Model.components.First_Plasmon_Peak.sigma.as_signal(field = 'values')
            self.FPP_Emax   = self.Fit_Model.components.First_Plasmon_Peak.centre.as_signal(field = 'values')
            self.FPP_Int    = self.Fit_Model.components.First_Plasmon_Peak.A.as_signal(field = 'values')

            self.SPP_FWHM   = 2*np.sqrt(np.log(2))*self.Fit_Model.components.Second_Plasmon_Peak.sigma.as_signal(field = 'values')
            self.SPP_Emax   = self.Fit_Model.components.Second_Plasmon_Peak.centre.as_signal(field = 'values')
            self.SPP_Int    = self.Fit_Model.components.Second_Plasmon_Peak.A.as_signal(field = 'values')

            self.Ep_q0      = ( self.FPP_Emax**2 - (self.FPP_FWHM / 2)**2 )**0.5
            
        elif (self.function_set == 'Voigt'):
        
            ZLP_L_fwhm      = self.Fit_Model.components.Zero_Loss_Peak.gamma.as_signal(field = 'values')
            ZLP_G_fwhm      = self.Fit_Model.components.Zero_Loss_Peak.FWHM.as_signal(field = 'values')
            self.ZLP_FWHM   = (ZLP_L_fwhm*0.5346 + 
                               np.sqrt(ZLP_G_fwhm**2 + ZLP_L_fwhm**2*0.2166)
                              )
            self.ZLP_Emax   = self.Fit_Model.components.Zero_Loss_Peak.centre.as_signal(field = 'values')
            self.ZLP_Int    = self.Fit_Model.components.Zero_Loss_Peak.area.as_signal(field = 'values')

            FPP_L_fwhm      = self.Fit_Model.components.First_Plasmon_Peak.gamma.as_signal(field = 'values')
            FPP_G_fwhm      = self.Fit_Model.components.First_Plasmon_Peak.FWHM.as_signal(field = 'values')
            self.FPP_FWHM   = (FPP_L_fwhm*0.5346 + 
                               np.sqrt(FPP_G_fwhm**2 + FPP_L_fwhm**2*0.2166)
                              )
            self.FPP_Emax   = self.Fit_Model.components.First_Plasmon_Peak.centre.as_signal(field = 'values')
            self.FPP_Int    = self.Fit_Model.components.First_Plasmon_Peak.area.as_signal(field = 'values')

            SPP_L_fwhm      = self.Fit_Model.components.Second_Plasmon_Peak.gamma.as_signal(field = 'values')
            SPP_G_fwhm      = self.Fit_Model.components.Second_Plasmon_Peak.FWHM.as_signal(field = 'values')
            self.SPP_FWHM   = (SPP_L_fwhm*0.5346 + 
                               np.sqrt(SPP_G_fwhm**2 + SPP_L_fwhm**2*0.2166)
                              )
            self.SPP_Emax   = self.Fit_Model.components.Second_Plasmon_Peak.centre.as_signal(field = 'values')
            self.SPP_Int    = self.Fit_Model.components.Second_Plasmon_Peak.area.as_signal(field = 'values')

            self.Ep_q0      = ( self.FPP_Emax**2 - (self.FPP_FWHM / 2)**2 )**0.5
        
        else:
            
            print('No valid function set specified. Please look into docstring for further information.')
        
        self.parameter_dict = {'$E_{max}$ - Zero Loss Peak'            : self.ZLP_Emax,
                               '$\Gamma$ - Zero Loss Peak'             : self.ZLP_FWHM,
                               'Intensity - Zero Loss Peak'            : self.ZLP_Int,

                               '$E_{max}$ - Plasmon Peak'              : self.FPP_Emax,
                               '$\Gamma$ - Plasmon Peak'               : self.FPP_FWHM,
                               'Intensity - Plasmon Peak'              : self.FPP_Int,

                               '$E_{max}$ - Second Order Plasmon Peak' : self.SPP_Emax,
                               '$\Gamma$ - Second Order Plasmon Peak'  : self.SPP_FWHM,
                               'Intensity - Second Order Plasmon Peak' : self.SPP_Int,

                               '$E_p(q=0)$ - Plasmon Energy'           : self.Ep_q0
                              }
        
        for title in self.parameter_dict:
            self.parameter_dict[title].metadata.General.title = title
    

    def plot_parameter_maps(self):
        """
        Visualize all parameter maps that were previously generated.
        """
        
        for title in self.parameter_dict:
            mean = self.parameter_dict[title].mean(axis=(0,1)).data[0]
            std  = self.parameter_dict[title].std(axis=(0,1)).data[0]
              
            self.parameter_dict[title].plot(vmin = mean - std, vmax = mean + std, scalebar_color='black')
    
    
    def print_stats(self):
        """
        Printing standard file information.
        """
        print('Statistics of loaded spectrum image:')
        self.File.print_summary_statistics()
        print('Statistics of normalized spectrum image:')
        self.File_norm.print_summary_statistics()
        print('Statistics of deconvolved spectrum image:')
        self.File_deconv.print_summary_statistics()
    
    def print_param_stats(self):
        """
        Printing standard parameter information
        """
        for title in self.parameter_dict:
            print(title)
            self.parameter_dict[title].print_summary_statistics()
    
    
    def create_dirs(self):
        """
        Creating directories and sub-directories for evaluation storage.
        """
        for directory in self.dir_list:
            if not os.path.exists(os.getcwd() + os.sep + directory):
                os.makedirs(os.getcwd() + os.sep + directory)
    
    
    def save_parameter_maps(self, dpi=300, fileformat='png'):
        """
        Saving function to save all generated parameter maps that were previously generated.
        """
        self.create_dirs()
        
        dir_fname = {'$E_{max}$ - Zero Loss Peak      [eV]'  : 'ZLP_Emax',
                     '$\Gamma$ - Zero Loss Peak       [eV]'  : 'ZLP_FWHM',
                     'Intensity - Zero Loss Peak'            : 'ZLP_Int' ,
                     
                     '$E_{max}$ - Plasmon Peak        [eV]'  : 'FPP_Emax',
                     '$\Gamma$ - Plasmon Peak         [eV]'  : 'FPP_FWHM',
                     'Intensity - Plasmon Peak'              : 'FPP_Int' ,
                     
                     '$E_{max}$ - Second Plasmon Peak [eV]'  : 'SPP_Emax',
                     '$\Gamma$ -  Second Plasmon Peak [eV]'  : 'SPP_FWHM',
                     'Intensity - Second Plasmon Peak'       : 'SPP_Int' ,
                     
                     '$E_p(q=0)$ - Plasmon Energy     [eV]'  : 'Ep_q0'
                    }
        
        for title in self.parameter_dict:

            fname = os.getcwd() + os.sep + self.dir_list[0] + os.sep + dir_fname[title]
            self.parameter_dict[title].metadata.General.title = title
            self.parameter_dict[title].plot(scalebar_color='black')
            plt.savefig(fname, dpi=dpi, extension=fileformat)
            plt.close()

        fname = os.getcwd() + os.sep + 'Model_Chisq'
        self.Chisq.metadata.General.title = 'Goodness of Fit: $\Chi ^2$'
        self.Chisq.plot(scalebar_color='black')
        plt.savefig(fname, dpi=dpi, extension=fileformat)
        plt.close()
        
        fname = os.getcwd() + os.sep + self.dir_list[0] + os.sep + 'Model_red_Chisq'
        self.red_Chisq.metadata.General.title = 'Goodness of Fit: $\Chi_{\nu} ^2$'
        self.red_Chisq.plot(scalebar_color='black')
        plt.savefig(fname, dpi=dpi, extension=fileformat)
        plt.close()
        
        fname = os.getcwd() + os.sep + self.dir_list[1] + os.sep + 'Foil_thickness'
        self.thickness_map.plot(scalebar_color='black')
        plt.savefig(fname, dpi=dpi, extension=fileformat)
        plt.close()
        
        
    # ROI setup for averaging multiple pixels to reduce calculation effort
    # due to reducing calculation of minimal chi-squared. 
    # This accounts for each pixel though only an average spectrum will be fitted
    # _width and _height will therefore specify the shape of the ROI for averaging
    def line_roi(self, param_map, x1=0, y1=0, x2=15, y2=15, width=0):
        """
        Comments missing
        """
        self.Line = hs.roi.Line2DROI(x1 * param_map.axes_manager[0].scale + 
                                     param_map.axes_manager[0].offset,
                                     y1 * param_map.axes_manager[1].scale + 
                                     param_map.axes_manager[1].offset,
                                     x2 * param_map.axes_manager[0].scale + 
                                     param_map.axes_manager[0].offset,
                                     y2 * param_map.axes_manager[1].scale + 
                                     param_map.axes_manager[1].offset,
                                     linewidth = width * param_map.axes_manager[0].scale
                                    )
        
        interactive = self.yes_or_no('Do you want to use the interactive gui?' + '\n' +
                                     'Adjusting position is possible using the interactive gui.'
                                    )
        
        if (interactive == True):
            param_map.plot(scalebar_color='black')
            self.Line_Scan = self.Line.interactive(param_map)
            
        else:
            self.Line_Scan = self.Line(param_map)

            
    def Line_Scan_std(self, param_map):
        angle = self.Line.angle()
        width = self.Line.linewidth
        
        line_std = self.Line_Scan.deepcopy()
        
        for i in range(len(self.Line_Scan.data)):
            print('next:')
            coord_x = self.Line.x1 + i * param_map.axes_manager[0].scale * np.cos(angle)
            coord_y = self.Line.y1 + i * param_map.axes_manager[0].scale * np.sin(angle)
            print(coord_x, coord_y)
            x1_std = coord_x + width/2 * np.sin(angle)
            y1_std = coord_y + width/2 * np.cos(angle)
            x2_std = coord_x - width/2 * np.sin(angle)
            y2_std = coord_y - width/2 * np.cos(angle)
            
            tmp_line = hs.roi.Line2DROI(x1_std,
                                        y1_std,
                                        x2_std,
                                        y2_std,
                                        linewidth = self.Line_Scan.axes_manager[0].scale
                                       )
            print(tmp_line)
            std_coords  = tmp_line(param_map).std(axis=0).data[0]
            print(std_coords)
            line_std.data[i] = std_coords
        
        #self.Line_Scan.std = line_std
        return line_std
            
    
    def rect_roi(self, param_map, left=0, top=0, right=15, bottom=15):
        """
        Comments missing
        """
        roi = hs.roi.RectangularROI(param_map.axes_manager.signal_axes[0].offset +
                                    left   * param_map.axes_manager.signal_axes[0].scale,
                                    param_map.axes_manager.signal_axes[1].offset +
                                    top    * param_map.axes_manager.signal_axes[1].scale,
                                    param_map.axes_manager.signal_axes[0].offset +
                                    right  * param_map.axes_manager.signal_axes[0].scale,
                                    param_map.axes_manager.signal_axes[1].offset +
                                    bottom * param_map.axes_manager.signal_axes[1].scale)
        interactive = self.yes_or_no('Do you want to use the interactive gui?' + '\n' +
                                     'Adjusting position is possible using the interactive gui.'
                                    )
        
        if (interactive == True):
            param_map.plot(scalebar_color='black')
            self.ROI = roi.interactive(param_map)
            
        else:
            self.ROI = roi(param_map)

    
    # Calculate thickness by log ratio of ZLP-Area to Total Area of EELS-data
    def calc_thickness(self, elements, concentrations):
        """
        Estimation of the sample thickness by the Log-Ratio method
                
            LATEX Formula:
                t = \lambda \ln{ \frac{I_t}{I_0} }
                
                with:
                         I_t = total intensity
                    and  I_0 = elastically scattered intensity
        
        For more information see:
            EELS Log-Ratio Technique for Specimen-Thickness
            Measurement in the TEM                    

            MALIS, S.C. CHENG, AND R.F. EGERTON
            JOURNAL OF ELECTRON MICROSCOPY TECHNIQUE 8:193-200 11988)
        
        Mean Free Path (\lambda) estimation is automated for a given elemental 
        composition.
        
        IMPORTANT:
            MFP-Automation takes metadata of file as input. Please verify that 
            values for beam energy and collection angle specified in .dm3/.dm4 
            files are correct before using this function.
        
        Currently supported elements: 
        (
         Ag, Al, Au, Be, Ca, Ce, Cu, Dy, Er, Eu, Fe, Gd, Ho, La, Lu, Mg, Nb, 
         Nd, Ni, P, Pd, Pm, Pr, Sm, Sn, Tb, Ti, Y, Yb, Zn, Zr
        )
        """
        t_lambda           = self.File.estimate_thickness(threshold=self.elastic_threshold).T
        mfp                = self.estimate_MFP(elements, concentrations)
        self.thickness_map = t_lambda * mfp
        
        self.thickness_map.metadata.General.title = r'thickness map $t = \lambda \cdot \ln{\frac{I_t}{I_0}}$' 
    
    
    def estimate_MFP(self, elements, concentrations):#, elements, concentration):
        """
        Estimation of the Mean Free Path for a given elemental composition.
        
        IMPORTANT:
            MFP-Automation takes metadata of file as input. Please verify that 
            values for beam energy and collection angle specified in .dm3/.dm4 
            files are correct before using this function.
            Default values (if not specified in metadata):
                E0   = 300 keV
                beta =  10 mrad
        
        Currently supported elements: 
        (
         Ag, Al, Au, Be, Ca, Ce, Cu, Dy, Er, Eu, Fe, Gd, Ho, La, Lu, Mg, Nb, 
         Nd, Ni, P, Pd, Pm, Pr, Sm, Sn, Tb, Ti, Y, Yb, Zn, Zr
        )
        """
        
        element_dict = {'Ag' : mend.Ag,
                        'Al' : mend.Au,
                        'Au' : mend.Au,
                        'Be' : mend.Be,
                        'Ca' : mend.Ca,
                        'Ce' : mend.Ce,
                        'Cu' : mend.Cu,
                        'Dy' : mend.Dy,
                        'Er' : mend.Er,
                        'Eu' : mend.Eu,
                        'Fe' : mend.Fe,
                        'Gd' : mend.Gd,
                        'Ho' : mend.Ho,
                        'La' : mend.La,
                        'Lu' : mend.Lu,
                        'Mg' : mend.Mg,
                        'Nb' : mend.Nb,
                        'Nd' : mend.Nd,
                        'Ni' : mend.Ni,
                        'P'  : mend.P,
                        'Pd' : mend.Pd,
                        'Pm' : mend.Pm,
                        'Pr' : mend.Pr,
                        'Pt' : mend.Pt,
                        'Sm' : mend.Sm,
                        'Sn' : mend.Sn,
                        'Tb' : mend.Tb,
                        'Ti' : mend.Ti,
                        'Tm' : mend.Tm,
                        'Y'  : mend.Y,
                        'Yb' : mend.Yb,
                        'Zn' : mend.Zn,
                        'Zr' : mend.Zr
                       }
        
        const              = 0.3 # Malis EELS paper - konstante r nach eq.(4)
        number_of_elements = len(elements)
        fi_Zi_numerator    = 0
        fi_Zi_denominator  = 0

        for i in range(number_of_elements):
            
            Z                  = element_dict[elements[i]].atomic_number
            fi_Zi_numerator   += concentrations[i] * Z**(1+const)
        
        
        for i in range(number_of_elements):
            
            Z                  = element_dict[elements[i]].atomic_number
            fi_Zi_denominator += concentrations[i] * Z**(const)

        Z_eff = fi_Zi_numerator/fi_Zi_denominator

        m    = 0.36 # Malis EELS paper - exponent m nach eq.(8)
        E_m  = 7.6*Z_eff**m # eq.(8): 7.6 eV

        try:
            E_0  = self.File.metadata.Acquisition_instrument.TEM.beam_energy
            
        except:
            print('No information on used beam energy found in metadata! Using default (300 keV).')
            E_0  = 300 # in keV
        
        try:
            beta = self.File.metadata.Acquisition_instrument.TEM.Detector.EELS.collection_angle
        
        except:
            print('No information on detector collection angle found in metadata! Using default (10 mrad).')
            beta = 10 # in mrad

        F = (1 + E_0/1022)/(1 + E_0/511)**2 # Malis EELS paper - E_0 in keV nach eq.(6) 

        mean_free_path = 106 * F * E_0 / (E_m * np.log(2 * beta * E_0 / E_m))
        
        return mean_free_path
    
    
    # Calculate the parametershifts between the gausfits of neighbouring pixels
    def plasmon_pshift(self):
        """
        Comments missing
        """
        return 0 #TO DO
   

    def watershed_algorithm(self):
        """
        Comments missing
        """
        return 0 # TO DO
 

    def filter_isolated_cells(array, struct):
        """
        Return array with completely isolated single cells removed

        :param array: Array with completely isolated single cells
        :param struct: Structure array for generating unique regions
        :return: Array with minimum region size > 1
        """
        filtered_array = np.copy(array)
        id_regions, num_ids = ndi.label(filtered_array, structure=struct)
        id_sizes = np.array(ndi.sum(array, id_regions, range(num_ids + 1)))
        area_mask = (id_sizes < 4)
        filtered_array[area_mask[id_regions]] = 0
        
        return filtered_array


    def process_image(data):
        """
        Comments missing
        """
        img_stack = mpimg.imread(data['input_file'], 0)
        image = img_stack[:, :, 0]

        # denoise image
        denoised_mark = rank.median(image, disk(data['denoised_mark_disk']))
        denoised_grad = rank.median(image, disk(data['denoised_grad_disk']))

        # find continuous region (low gradient -
        # where less than 8 for this image) --> markers
        # disk(10) is used here to get a more smooth image
        markers_binary = rank.gradient(
            denoised_mark, disk(data['markers_binary_disk'])) \
            < data['markers_binary_min_gradient']

        # Extra Filter for dilation and erosion
        # Dilations to increase big particles
        markers_dilated = binary_dilation(
            markers_binary, diamond(data['markers_dilated_diamond'])
        ).astype(np.uint8)
        markers_erosion = binary_erosion(
            markers_dilated, diamond(data['markers_erosion_diamond'])
        ).astype(np.uint8)

        # Median to get rid of small markers after erosion
        markers_denoised = rank.median(
            markers_erosion, disk(data['markers_denoised_disk']))

        # Run function on sample array to filter last
        # isolated cells
        markers_filtered = filter_isolated_cells(
            markers_denoised, struct=np.ones((3, 3)))
        # Run another dilated function call to try to
        # connect small cells
        markers_last = binary_dilation(
            markers_filtered, diamond(data['markers_last_diamond'])
        ).astype(np.uint8)

        # Label markers
        markers_labeled, _ = ndi.label(markers_last)
        markers = markers_labeled

        # local gradient (disk(2) is used to keep edges thin)
        gradient_binary = rank.gradient(
            denoised_grad, disk(data['gradient_binary_disk'])
        ) > data['gradient_binary_min_gradient']
        gradient_dilated = binary_dilation(
            gradient_binary, diamond(data['gradient_dilated_diamond'])
        ).astype(np.uint8)
        gradient = gradient_dilated

        # process the watershed
        labels = watershed(gradient, markers)

        # save number of grains
        markers_array, num_features = ndi.label(markers_labeled)
        Grainsize = np.empty(num_features)
        for label in range(0, num_features):
            size_pixel = np.sum(np.where(markers_array == label + 1, 1, 0))
            size_mm2 = size_pixel * data['mm_per_pixel']**2
            Grainsize[label] = size_mm2

        avg_gs = np.mean(Grainsize)
        avg_gr = 2 * (avg_gs / np.pi)**0.5


In [3]:
l.split('asdf_asdfg', ('_'))[0]

NameError: name 'l' is not defined

In [32]:
im.axes_manager.navigation_shape[0]

200

In [5]:
im = hs.load(filenames='18.01.26 Spectrum image EELS.dm3')
im.metadata#.axes_manager

├── General
│   ├── original_filename = 18.01.26 Spectrum image EELS.dm3
│   └── title = 18.01.26 Spectrum image EELS
└── Signal
    ├── Noise_properties
    │   └── Variance_linear_model
    │       ├── gain_factor = 1.0
    │       └── gain_offset = 0.0
    ├── binned = True
    ├── quantity = Intensity
    └── signal_type = EELS

In [6]:
m=im.create_model(auto_background=False)
g=hyperspy.components1d.Lorentzian(A=10000,centre=15)
m.append(g)
m.set_signal_range(10.,30.)
m.multifit()
m.plot()

In [7]:
area = m.components.Lorentzian.gamma.as_signal(field='values')

In [8]:
area.axes_manager.signal_axes[1].scale = 1.0

In [9]:
area.axes_manager.signal_axes[0].offset = 0
area.axes_manager.signal_axes[0].offset = 0

In [115]:
left=0
top=0
right=100
bottom=50

roi = hs.roi.RectangularROI(area.axes_manager.signal_axes[0].offset+left*area.axes_manager.signal_axes[0].scale,
                            area.axes_manager.signal_axes[1].offset+top*area.axes_manager.signal_axes[1].scale,
                            area.axes_manager.signal_axes[0].offset+right*area.axes_manager.signal_axes[0].scale,
                            area.axes_manager.signal_axes[1].offset+bottom*area.axes_manager.signal_axes[1].scale)
#roi.linewidth = 3
area.plot()
roi_area = roi.interactive(area)

In [11]:
roi_area

<Signal2D, title: gamma parameter of Lorentzian component, dimensions: (|100, 50)>

In [11]:
roi_area.plot()

In [122]:
import cv2

img=roi_area.data
#img=cv2.imread('Figure_intensity_parameter_of_First_Plasmon_Peak_component_Signal.png') 

img=255*(1-(img-np.min(img))/(np.max(img)-np.min(img)))
#img=255*(img-np.min(img))/(np.max(img)-np.min(img))

img_gauss = cv2.GaussianBlur(img, (5,5), cv2.BORDER_DEFAULT)
img_gauss=img_gauss.astype('uint8')
vis = img_gauss.copy()
mser = cv2.MSER_create()
coordinates, bboxes = mser.detectRegions(img_gauss)

hulls = [cv2.convexHull(p.reshape(-1, 1, 2)) for p in regions]
cv2.polylines(img_gauss, hulls, 1, (0, 0, 255), 2)

FD = cv2.Feature2D('MSER')
kpoints = mser.detect(img_gauss)

mask = np.zeros((img.shape[0], img.shape[1], 1), dtype=np.uint8)

for contour in hulls:

    cv2.drawContours(mask, [contour], -1, (255, 255, 255), -1)
    
sb_only = cv2.bitwise_and(img_gauss, img_gauss, mask=mask)


coords = []
for coord in coordinates:
    bbox = cv2.boundingRect(coord)
    x,y,w,h = bbox
    if w< 10 or h < 10 or w/h > 5 or h/w > 5:
        continue
    coords.append(coord)

In [110]:

"""
colors = [[43, 43, 200], 
          [43, 75, 200], 
          [43, 106, 200], 
          [43, 137, 200], 
          [43, 169, 200], 
          [43, 200, 195], 
          [43, 200, 163], 
          [43, 200, 132], 
          [43, 200, 101], 
          [43, 200, 69], 
          [54, 200, 43], 
          [85, 200, 43], 
          [116, 200, 43], 
          [148, 200, 43], 
          [179, 200, 43], 
          [200, 184, 43], 
          [200, 153, 43], 
          [200, 122, 43], 
          [200, 90, 43], 
          [200, 59, 43], 
          [200, 43, 64], 
          [200, 43, 95], 
          [200, 43, 127], 
          [200, 43, 158], 
          [200, 43, 190], 
          [174, 43, 200], 
          [142, 43, 200], 
          [111, 43, 200], 
          [80, 43, 200], 
          [43, 43, 200]
         ]
"""

## Fill with random colors
np.random.seed(0)
canvas1 = img_gauss.copy()
canvas2 = cv2.cvtColor(img_gauss, cv2.COLOR_GRAY2BGR)
canvas3 = np.zeros_like(canvas2)

for cnt in coords:
    xx = cnt[:,0]
    yy = cnt[:,1]
    color = colors[np.random.choice(len(colors))]
    #canvas1[yy, xx] = color
    canvas2[yy, xx] = color
    canvas3[yy, xx] = color

plt.imshow(canvas3)

In [98]:
print(np.max(np.transpose(coords[0])[0]))
print(np.min(np.transpose(coords[0])[0]))

157
1


In [99]:
print(np.max(np.transpose(coords[0])[1]))
print(np.min(np.transpose(coords[0])[1]))

40
1


In [145]:
for i in range(len(kpoints)):
    print('Coordinates: %f, %f' %(int(kpoints[i].pt[0]), int(kpoints[i].pt[1])))

Coordinates: 45.000000, 20.000000
Coordinates: 45.000000, 20.000000
Coordinates: 45.000000, 20.000000
Coordinates: 45.000000, 20.000000
Coordinates: 45.000000, 20.000000
Coordinates: 45.000000, 20.000000
Coordinates: 46.000000, 20.000000
Coordinates: 47.000000, 20.000000
Coordinates: 46.000000, 20.000000
Coordinates: 46.000000, 20.000000
Coordinates: 38.000000, 19.000000
Coordinates: 40.000000, 24.000000
Coordinates: 60.000000, 44.000000
Coordinates: 40.000000, 44.000000
Coordinates: 42.000000, 20.000000
Coordinates: 42.000000, 20.000000
Coordinates: 42.000000, 20.000000
Coordinates: 42.000000, 20.000000
Coordinates: 42.000000, 20.000000
Coordinates: 41.000000, 20.000000
Coordinates: 40.000000, 24.000000


In [146]:
#plt.imshow(vis)
cv2.imshow('img', vis)

#cv2.waitKey(0)

#cv2.destroyAllWindows()

In [43]:
regions[]

[array([[  3,   3],
        [  4,   5],
        [  5,   5],
        ...,
        [130,  23],
        [155,  15],
        [111,  17]], dtype=int32), array([[  3,   3],
        [  4,   5],
        [  5,   5],
        ...,
        [131,  17],
        [141,  22],
        [112,  23]], dtype=int32), array([[  3,   3],
        [  4,   5],
        [  5,   5],
        ...,
        [102,  19],
        [117,  17],
        [140,  22]], dtype=int32), array([[  3,   3],
        [  4,   5],
        [  5,   5],
        ...,
        [ 94,  24],
        [134,  17],
        [133,  23]], dtype=int32), array([[  3,   3],
        [  4,   5],
        [  5,   5],
        ...,
        [136,  22],
        [122,  22],
        [ 96,  20]], dtype=int32), array([[  3,   3],
        [  4,   5],
        [  5,   5],
        ...,
        [127,  18],
        [112,  22],
        [115,  21]], dtype=int32), array([[  3,   3],
        [  4,   5],
        [  5,   5],
        ...,
        [114,  21],
        [100,  23],
     

In [41]:
plt.imshow(img_gauss)

In [7]:
file=hs.load('../Master/EELS AlYFe2/Energy Drift Corr from AlyFe sb3 EELS SI_1.dm3')

In [34]:
test = file.axes_manager.as_dictionary()['axis-0']

In [36]:
test['units']

'eV'

# Testing _STZ-Mapping_ Routine

In [64]:
a=None
a=Plasmon_mapper(file=hs.load('../Master/EELS-SB1-Paper/EELS Spectrum Image (Energy shift (Gaussian) corrected).dm3'), is_lazy=True)
a.File.metadata.General.title = 'Vitreloy SB3'
#a = STZ_mapper(filename = '18_01_26_Spectrum_image_14Plasmon_FWHM.dm3', use_roi=False)

#m = a.eels_fit_routine()
#m.plot()
#m = im[0].create_model()
#m.print_current_values(only_free=False)
#print(m.red_chisq.data)
#print(m.components)
#m.components.Zero_Loss_Peak.function(x=-0.270658)

Does the file include stored models from previous fitting? (y/n): n
Setting up proper navigation space...
Energy loss axis
Aligning Zero-Loss Peak...

Initial ZLP position statistics
-------------------------------
Summary statistics
------------------
mean:	-1.61
std:	0.0593

min:	-1.7
Q1:	-1.6
median:	-1.6
Q3:	-1.6
max:	-1.1


Aligning datastructure successful. Estimate poissonian noise...
Do you want to continue with the deconvoluted spectrum using
the Fourier-Log-Deconvolution method?
[Less calculation effort in the fitting routine as only the
 plasmon peak is fitted. Precision might decrease.
 (y/n): n
Loading process completed. Energy channels cropped by ZLP-Alignment: 9[0.439453125%]
Channels before: 2048
Channels after: 2039


In [ ]:
a=None
a=Plasmon_mapper(file=hs.load('EELS Low_Loss_Models.hspy'), is_lazy=True)

In [59]:
a.File.plot()

In [12]:
a.calc_thickness(['Cu','Zr'], [0.64,0.36])

In [13]:
a.thickness_map.plot()

In [7]:
a.File.plot()

In [5]:
a.File.metadata.General.title = 'EELS Spectrum Image 1 4nm-probe doubleSBarea'
a.File.plot()

In [122]:
a.function_set = 'VolumePlasmonDrude'
m = a.init_model(False, False)
a.fit_zlp_only(m, 'leastsq', 'ls', True)
a.fit_pp_only(100., m, 'leastsq', 'ls', True)

Do you want to use the gui to manually set the starting parameters? (y/n): n


12.840158282976358

In [129]:
m.plot(plot_components=True)

/home/max/Apps/anaconda3/envs/Hyperspy/lib/python3.6/site-packages/matplotlib/tight_layout.py:176: UserWarning: Tight layout not applied. The left and right margins cannot be made large enough to accommodate all axes decorations. 
  warnings.warn('Tight layout not applied. The left and right margins '


In [117]:
test=a.Fit_Model

In [124]:
a.set_model_params(test, m, func='First_Plasmon_Peak')

In [127]:
test.set_parameters_value('plasmon_energy',
                          m.components.First_Plasmon_Peak.plasmon_energy.value,
                          component_list=['First_Plasmon_Peak']
                         )

In [128]:
test.components.First_Plasmon_Peak.plasmon_energy.map['values']

array([[104.09730699],
       [104.09730699],
       [104.09730699],
       [104.09730699],
       [104.09730699]])

In [115]:
a.Fit_Model.components

AttributeError: 'NoneType' object has no attribute 'components'

In [63]:
m.print_current_values()

Components	Parameter	Value
Zero_Loss_Peak
		FWHM	1.58797
		area	395135
		centre	-0.303945
		gamma	0.689704
First_Plasmon_Peak
		A	64999.5
		centre	16.0222
		gamma	2.92928
Second_Plasmon_Peak
		A	764383
		gamma	38.1443


In [47]:
a.Fit_Model.print_current_values()

Components	Parameter	Value
Zero_Loss_Peak
		FWHM	2.31046
		area	370134
		centre	-0.46832
		gamma	0.358628
First_Plasmon_Peak
		fwhm	-7.03859e+06
		intensity	7.65043e+06
		plasmon_energy	6548.99
Second_Plasmon_Peak
		fwhm	2.80828e+07
		intensity	7.65802e+06


In [162]:
a.Fit_Model.plot_results(only_active=False)

   # |      Attribute Name |      Component Name |      Component Type
---- | ------------------- | ------------------- | -------------------
   0 |      Zero_Loss_Peak |      Zero_Loss_Peak |               Voigt
   1 |  First_Plasmon_Peak |  First_Plasmon_Peak |  VolumePlasmonDrude
   2 | Second_Plasmon_Peak | Second_Plasmon_Peak |  VolumePlasmonDrude

In [40]:
a.load_model()

Available models: 
└── VolumePlasmonDrude_ls
    ├── components
    │   ├── First_Plasmon_Peak
    │   ├── Second_Plasmon_Peak
    │   └── Zero_Loss_Peak
    ├── date = 2019-06-25 16:02:29
    └── dimensions = (1, 5|2043)

Model names: 
{'1': 'VolumePlasmonDrude_ls', '2': 'VolumePlasmonDrude_ml', '3': 'Lorentzian_ls', '4': 'Lorentzian_ml', '5': 'Gaussian_ls', '6': 'Gaussian_ml', '7': 'Voigt_ls', '8': 'Voigt_ml'}
Which model should be loaded? 1
Loading parameter maps for: VolumePlasmonDrude_ls


/home/max/Apps/anaconda3/envs/Hyperspy/lib/python3.6/site-packages/hyperspy/signal.py:1696: RuntimeWarning: invalid value encountered in sqrt
  getattr(self.data, op_name)(other))


In [76]:
a.FPP_Emax.plot(vmin=ep-std, vmax=ep+std)

In [74]:
ep=a.FPP_Emax.mean(axis=(0,1)).data[0]

In [75]:
std=a.FPP_Emax.std(axis=(0,1)).data[0]

In [11]:
a.generate_param_maps()

No valid function set specified. Please look into docstring for further information.


AttributeError: 'NoneType' object has no attribute 'metadata'

In [26]:
a.FPP_Int.plot()

In [32]:
a.plot_parameter_maps()

In [62]:
a.Fit_Model.as_signal().plot()

In [140]:
a.Fit_Model.plot(plot_components=True)

/home/max/Apps/anaconda3/envs/Hyperspy/lib/python3.6/site-packages/matplotlib/tight_layout.py:176: UserWarning: Tight layout not applied. The left and right margins cannot be made large enough to accommodate all axes decorations. 
  warnings.warn('Tight layout not applied. The left and right margins '


In [48]:
a.Fit_Model.print_current_values()

Components	Parameter	Value
Zero_Loss_Peak
		FWHM	1.11969
		area	418014
		centre	-0.234404
		gamma	0.809662
First_Plasmon_Peak
		fwhm	60.7358
		intensity	2273.24
		plasmon_energy	28.4308
Second_Plasmon_Peak
		fwhm	1.60603e+29
		intensity	2.61015e+28


In [26]:
a.File.mean().plot()

In [125]:
a.fit_fpp_only(a.Fit_Model, 'leastsq', 'ls')

In [50]:
test = a.rect_roi(a.Fit_Model.as_signal(), 100,100,150,150)

IndexError: tuple index out of range

In [19]:
a.Fit_Model.plot(plot_components=True)

/home/max/Apps/anaconda3/envs/Hyperspy/lib/python3.6/site-packages/matplotlib/tight_layout.py:176: UserWarning: Tight layout not applied. The left and right margins cannot be made large enough to accommodate all axes decorations. 
  warnings.warn('Tight layout not applied. The left and right margins '


In [16]:
a.Fit_Model.axes_manager.indices = (1,1)
a.Fit_Model.plot(plot_components=True)

TraitError: The 'index' trait of a DataAxis instance must be 0 <= a number <= 0, but a value of 1 <class 'int'> was specified.

In [40]:
a.Fit_Model.red_chisq.plot()

In [60]:
#a.Fit_Model.axes_manager.indices = (5,5)
a.Fit_Model.print_current_values()

Components	Parameter	Value
Zero_Loss_Peak
		FWHM	2.32304
		area	363077
		centre	-0.493488
		gamma	0.327345
First_Plasmon_Peak
		fwhm	-2339.81
		intensity	1.69324e+07
		plasmon_energy	137.583
Second_Plasmon_Peak
		fwhm	9326.58
		intensity	1.69872e+07


In [171]:
m=a.File.create_model()
g=hyperspy.components1d.Gaussian()
m.extend([g])
m.components

   # |      Attribute Name |      Component Name |      Component Type
---- | ------------------- | ------------------- | -------------------
   0 |            PowerLaw |            PowerLaw |            PowerLaw
   1 |            Gaussian |            Gaussian |            Gaussian

In [69]:
a.Fit_Model.plot(plot_components=True)

In [70]:
a.Fit_Model.red_chisq.plot()

In [71]:
a.plot_parameter_maps()

In [65]:
a.eels_fit_routine(function_set='VolumePlasmonDrude', samfire=True, multithreading=False, workers=8)

Do you want to use the gui to manually set the starting parameters? (y/n): n


Model will be stored in file...
Stored models in file:
└── VolumePlasmonDrude_ls
    ├── components
    │   ├── First_Plasmon_Peak
    │   ├── Second_Plasmon_Peak
    │   └── Zero_Loss_Peak
    ├── date = 2019-07-07 11:17:57
    └── dimensions = (62, 503|2039)



In [139]:
a.eels_fit_routine(function_set='VolumePlasmonDrude')

Do you want to use the gui to manually set the starting parameters? (y/n): n
Components	Parameter	Value
Zero_Loss_Peak
		FWHM	0.1
		area	372139
		centre	0
		gamma	0.1
First_Plasmon_Peak
		fwhm	0.1
		intensity	18606.9
		plasmon_energy	15
Second_Plasmon_Peak
		fwhm	0.1
		intensity	1860.69
Components	Parameter	Value
Zero_Loss_Peak
		FWHM	1.588
		area	395134
		centre	-0.303949
		gamma	0.689692
First_Plasmon_Peak
		fwhm	19.2584
		intensity	354.342
		plasmon_energy	20.2953
Second_Plasmon_Peak
		fwhm	100
		intensity	14164.3


Components	Parameter	Value
Zero_Loss_Peak
		FWHM	2.46857
		area	350948
		centre	-0.538735
		gamma	0.230404
First_Plasmon_Peak
		fwhm	33.2841
		intensity	9501.19
		plasmon_energy	22.4703
Second_Plasmon_Peak
		fwhm	26.4352
		intensity	2969.12
Model will be stored in file...
Stored models in file:
└── VolumePlasmonDrude_ls
    ├── components
    │   ├── First_Plasmon_Peak
    │   ├── Second_Plasmon_Peak
    │   └── Zero_Loss_Peak
    ├── date = 2019-06-25 18:31:43
    └── dimensions = (1, 5|2043)



In [18]:
a.eels_fit_routine(function_set='Lorentzian', samfire=True, multithreading=False)

Do you want to use the gui to manually set the starting parameters? (y/n): n


Model will be stored in file...
Stored models in file:
├── Lorentzian_ls
│   ├── components
│   │   ├── First_Plasmon_Peak
│   │   ├── Second_Plasmon_Peak
│   │   └── Zero_Loss_Peak
│   ├── date = 2019-06-26 12:24:15
│   └── dimensions = (1, 5|2043)
└── VolumePlasmonDrude_ls
    ├── components
    │   ├── First_Plasmon_Peak
    │   ├── Second_Plasmon_Peak
    │   └── Zero_Loss_Peak
    ├── date = 2019-06-26 12:23:24
    └── dimensions = (1, 5|2043)



In [58]:
a.eels_fit_routine(function_set='Gaussian', samfire=True)

Do you want to use the gui to manually set the starting parameters? (y/n): n
Estimate function parameters...


/home/max/Apps/anaconda3/envs/Hyperspy/lib/python3.6/site-packages/hyperspy/_components/voigt.py:202: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  data = signal.data[data_gi]
/home/max/Apps/anaconda3/envs/Hyperspy/lib/python3.6/site-packages/hyperspy/_components/voigt.py:61: RuntimeWarning: invalid value encountered in true_divide
  V = wofz(z) / (math.sqrt(2 * np.pi) * sigma)


/home/max/Apps/anaconda3/envs/Hyperspy/lib/python3.6/site-packages/hyperspy/model.py:1131: RuntimeWarning: invalid value encountered in sqrt
  self.p_std = np.sqrt(np.diag(pcov))
/home/max/Apps/anaconda3/envs/Hyperspy/lib/python3.6/site-packages/hyperspy/model.py:1131: RuntimeWarning: invalid value encountered in sqrt
  self.p_std = np.sqrt(np.diag(pcov))
/home/max/Apps/anaconda3/envs/Hyperspy/lib/python3.6/site-packages/hyperspy/model.py:1131: RuntimeWarning: invalid value encountered in sqrt
  self.p_std = np.sqrt(np.diag(pcov))


Model will be stored in file...
Stored models in file:
├── Gaussian_ls
│   ├── components
│   │   ├── First_Plasmon_Peak
│   │   ├── Second_Plasmon_Peak
│   │   └── Zero_Loss_Peak
│   ├── date = 2019-06-11 12:11:08
│   └── dimensions = (26, 26|2044)
├── Lorentzian_ls
│   ├── components
│   │   ├── First_Plasmon_Peak
│   │   ├── Second_Plasmon_Peak
│   │   └── Zero_Loss_Peak
│   ├── date = 2019-06-11 12:06:18
│   └── dimensions = (26, 26|2044)
└── VolumePlasmonDrude_ls
    ├── components
    │   ├── First_Plasmon_Peak
    │   ├── Second_Plasmon_Peak
    │   └── Zero_Loss_Peak
    ├── date = 2019-06-11 12:03:47
    └── dimensions = (26, 26|2044)



In [ ]:
a.eels_fit_routine(function_set='Voigt', samfire=True)

In [ ]:
a.eels_fit_routine(function_set='Gaussian', fitter='Nelder-Mead', method='ml', samfire=True)

In [87]:
a.eels_fit_routine(function_set='Lorentzian', fitter='Nelder-Mead', method='ml', samfire=True)
#a.store_model('LORENTZIAN_nm', 'lorentzian_nm')

/home/max/Apps/anaconda3/envs/Hyperspy/lib/python3.6/site-packages/hyperspy/_components/voigt.py:61: RuntimeWarning: overflow encountered in true_divide
  V = wofz(z) / (math.sqrt(2 * np.pi) * sigma)
/home/max/Apps/anaconda3/envs/Hyperspy/lib/python3.6/site-packages/hyperspy/_components/voigt.py:62: RuntimeWarning: overflow encountered in multiply
  return scale * V.real


<Axes manager, axes: (200, 50|2042)>
            Name |   size |  index |  offset |   scale |  units 
================ | ====== | ====== | ======= | ======= | ====== 
               x |    200 |      9 |       0 |       1 |     nm 
               y |     50 |      9 |       0 |       1 |     nm 
---------------- | ------ | ------ | ------- | ------- | ------ 
     Energy loss |   2042 |        |     -20 |     0.1 |     eV 
<Axes manager, axes: (200, 50|2042)>
            Name |   size |  index |  offset |   scale |  units 
================ | ====== | ====== | ======= | ======= | ====== 
               x |    200 |      9 |       0 |       1 |     nm 
               y |     50 |      9 |       0 |       1 |     nm 
---------------- | ------ | ------ | ------- | ------- | ------ 
     Energy loss |   2042 |        |     -20 |     0.1 |     eV 
Model will be stored in file...
Stored models in file:
├── Lorentzian_NelderMead
│   ├── components
│   │   ├── First_Plasmon_Peak
│   │   ├── Sec

Traceback (most recent call last):
  File "/home/max/Apps/anaconda3/envs/Hyperspy/lib/python3.6/site-packages/matplotlib/cbook/__init__.py", line 215, in process
    func(*args, **kwargs)
  File "/home/max/Apps/anaconda3/envs/Hyperspy/lib/python3.6/site-packages/hyperspy/drawing/utils.py", line 171, in function_wrapper
    function()
  File "/home/max/Apps/anaconda3/envs/Hyperspy/lib/python3.6/site-packages/hyperspy/drawing/figure.py", line 101, in _on_close
    for marker in self.ax_markers:
AttributeError: 'HistogramTilePlot' object has no attribute 'ax_markers'


In [90]:
a.eels_fit_routine(function_set='Voigt', samfire=True, fitter='Nelder-Mead', method='ml')
#a.store_model('VOIGT_ls', 'voigt_ls')

Estimate function parameters...


Traceback (most recent call last):
  File "/home/max/Apps/anaconda3/envs/Hyperspy/lib/python3.6/site-packages/matplotlib/cbook/__init__.py", line 215, in process
    func(*args, **kwargs)
  File "/home/max/Apps/anaconda3/envs/Hyperspy/lib/python3.6/site-packages/hyperspy/drawing/utils.py", line 171, in function_wrapper
    function()
  File "/home/max/Apps/anaconda3/envs/Hyperspy/lib/python3.6/site-packages/hyperspy/drawing/figure.py", line 101, in _on_close
    for marker in self.ax_markers:
AttributeError: 'HistogramTilePlot' object has no attribute 'ax_markers'


<Axes manager, axes: (200, 50|2042)>
            Name |   size |  index |  offset |   scale |  units 
================ | ====== | ====== | ======= | ======= | ====== 
               x |    200 |      9 |       0 |       1 |     nm 
               y |     50 |      9 |       0 |       1 |     nm 
---------------- | ------ | ------ | ------- | ------- | ------ 
     Energy loss |   2042 |        |     -20 |     0.1 |     eV 
<Axes manager, axes: (200, 50|2042)>
            Name |   size |  index |  offset |   scale |  units 
================ | ====== | ====== | ======= | ======= | ====== 
               x |    200 |      9 |       0 |       1 |     nm 
               y |     50 |      9 |       0 |       1 |     nm 
---------------- | ------ | ------ | ------- | ------- | ------ 
     Energy loss |   2042 |        |     -20 |     0.1 |     eV 
Model will be stored in file...
Stored models in file:
├── Gaussian_NelderMead
│   ├── components
│   │   ├── First_Plasmon_Peak
│   │   ├── Secon

In [16]:
a.Fit_Model.plot(plot_components=True)

In [17]:
a.rect_roi(a.FPP_Emax, 100, 100, 150, 150)

Do you want to use the interactive gui?
Adjusting position is possible using the interactive gui. (y/n): y


NameError: name 'area' is not defined

In [22]:
a.generate_param_maps()

In [23]:
a.plot_parameter_maps()

In [14]:
a.File.save('18.01.26 Spectrum image EELS Low_Loss_Models.hspy')

In [15]:
a.load_model()

Available models: 
└── VolumePlasmonDrude_ls
    ├── components
    │   ├── First_Plasmon_Peak
    │   ├── Second_Plasmon_Peak
    │   └── Zero_Loss_Peak
    ├── date = 2019-06-14 11:27:54
    └── dimensions = (136, 274|2027)

Model names: 
{'1': 'VolumePlasmonDrude_ls', '2': 'VolumePlasmonDrude_ml', '3': 'Lorentzian_ls', '4': 'Lorentzian_ml', '5': 'Gaussian_ls', '6': 'Gaussian_ml', '7': 'Voigt_ls', '8': 'Voigt_ml'}
Which model should be loaded? 1


In [8]:
a.Fit_Model.plot(plot_components=True)

In [24]:
l=None
l=Plasmon_mapper(file=hs.load('18.01.26 Spectrum image EELS Low_Loss_Models.hspy'))

Does the file include stored models from previous fitting? (y/n): y
Available models: 
└── VolumePlasmonDrude_ls
    ├── components
    │   ├── First_Plasmon_Peak
    │   ├── Second_Plasmon_Peak
    │   └── Zero_Loss_Peak
    ├── date = 2019-06-06 08:33:27
    └── dimensions = (200, 50|2042)

Model names: 
{'1': 'VolumePlasmonDrude_ls', '2': 'VolumePlasmonDrude_ml', '3': 'Lorentzian_ls', '4': 'Lorentzian_ml', '5': 'Gaussian_ls', '6': 'Gaussian_ml', '7': 'Voigt_ls', '8': 'Voigt_ml'}
Which model should be loaded? 1


In [36]:
l.eels_fit_routine(function_set='Lorentzian', fitter='Nelder-Mead', method='ml', samfire=True)

Do you want to use the gui to manually set the starting parameters? (y/n): n


/home/max/Apps/anaconda3/envs/Hyperspy/lib/python3.6/site-packages/hyperspy/_components/voigt.py:61: RuntimeWarning: invalid value encountered in true_divide
  V = wofz(z) / (math.sqrt(2 * np.pi) * sigma)


Model will be stored in file...
Stored models in file:
├── Lorentzian_ls
│   ├── components
│   │   ├── First_Plasmon_Peak
│   │   ├── Second_Plasmon_Peak
│   │   └── Zero_Loss_Peak
│   ├── date = 2019-06-06 13:11:43
│   └── dimensions = (200, 50|2042)
├── Lorentzian_ml
│   ├── components
│   │   ├── First_Plasmon_Peak
│   │   ├── Second_Plasmon_Peak
│   │   └── Zero_Loss_Peak
│   ├── date = 2019-06-06 14:08:22
│   └── dimensions = (200, 50|2042)
├── VolumePlasmonDrude_ls
│   ├── components
│   │   ├── First_Plasmon_Peak
│   │   ├── Second_Plasmon_Peak
│   │   └── Zero_Loss_Peak
│   ├── date = 2019-06-06 08:33:27
│   └── dimensions = (200, 50|2042)
└── VolumePlasmonDrude_ml
    ├── components
    │   ├── First_Plasmon_Peak
    │   ├── Second_Plasmon_Peak
    │   └── Zero_Loss_Peak
    ├── date = 2019-06-06 13:00:23
    └── dimensions = (200, 50|2042)



/home/max/Apps/anaconda3/envs/Hyperspy/lib/python3.6/site-packages/hyperspy/signal.py:1696: RuntimeWarning: invalid value encountered in sqrt
  getattr(self.data, op_name)(other))


In [31]:
l.load_model()

Available models: 
├── VolumePlasmonDrude_ls
│   ├── components
│   │   ├── First_Plasmon_Peak
│   │   ├── Second_Plasmon_Peak
│   │   └── Zero_Loss_Peak
│   ├── date = 2019-06-06 08:33:27
│   └── dimensions = (200, 50|2042)
└── VolumePlasmonDrude_ml
    ├── components
    │   ├── First_Plasmon_Peak
    │   ├── Second_Plasmon_Peak
    │   └── Zero_Loss_Peak
    ├── date = 2019-06-06 13:00:23
    └── dimensions = (200, 50|2042)

Model names: 
{'1': 'VolumePlasmonDrude_ls', '2': 'VolumePlasmonDrude_ml', '3': 'Lorentzian_ls', '4': 'Lorentzian_ml', '5': 'Gaussian_ls', '6': 'Gaussian_ml', '7': 'Voigt_ls', '8': 'Voigt_ml'}
Which model should be loaded? 1


In [39]:
l.Fit_Model.plot(plot_components=True)

In [38]:
l.Fit_Model.red_chisq.plot()

In [34]:
l.save_models_to_file('18.01.26 Spectrum image EELS Low_Loss_Models.hspy')

Overwrite '18.01.26 Spectrum image EELS Low_Loss_Models.hspy' (y/n)?
y


In [37]:
l.Fit_Model.red_chisq.print_summary_statistics()

Summary statistics
------------------
mean:	55.4
std:	57.9

min:	30.1
Q1:	39.4
median:	41.8
Q3:	44.6
max:	2e+03


In [30]:
l.Fit_Model.red_chisq.print_summary_statistics()

Summary statistics
------------------
mean:	146
std:	98.7

min:	96.8
Q1:	131
median:	140
Q3:	149
max:	5.86e+03


In [9]:
l.generate_param_maps()

In [10]:
l.plot_parameter_maps()

In [27]:
l.Fit_Model.plot(plot_components=True)

In [17]:
l.Fit_Model.fit(fitter='Nelder-Mead', method='ml')

/home/max/Apps/anaconda3/envs/Hyperspy/lib/python3.6/site-packages/hyperspy/_components/voigt.py:61: RuntimeWarning: invalid value encountered in true_divide
  V = wofz(z) / (math.sqrt(2 * np.pi) * sigma)


In [10]:
l.Fit_Model.plot(plot_components=True)

In [216]:
l.Fit_Model.red_chisq.plot()

In [222]:
l.File.axes_manager

<Axes manager, axes: (10, 10|2044)>
            Name |   size |  index |  offset |   scale |  units 
================ | ====== | ====== | ======= | ======= | ====== 
               x |     10 |      0 |       0 |       1 |     nm 
               y |     10 |      0 |       0 |       1 |     nm 
---------------- | ------ | ------ | ------- | ------- | ------ 
     Energy loss |   2044 |        |     -20 |     0.1 |     eV

In [204]:
l.print_file_information()

├── General
│   ├── original_filename = 18.01.26 Spectrum image EELS.dm3
│   └── title = 18.01.26 Spectrum image EELS
└── Signal
    ├── Noise_properties
    │   ├── Variance_linear_model
    │   │   ├── correlation_factor = 1
    │   │   ├── gain_factor = 1.0
    │   │   └── gain_offset = 0.0
    │   └── variance = <Signal1D, title: Variance of 18.01.26 Spectrum image EELS, dimensions: (5, 5|2044)>
    ├── binned = False
    ├── quantity = Intensity
    └── signal_type = EELS

<Axes manager, axes: (5, 5|2044)>
            Name |   size |  index |  offset |   scale |  units 
================ | ====== | ====== | ======= | ======= | ====== 
               x |      5 |      0 |       0 |       1 |     nm 
               y |      5 |      0 |       0 |       1 |     nm 
---------------- | ------ | ------ | ------- | ------- | ------ 
     Energy loss |   2044 |        |     -20 |     0.1 |     eV 
├── Gaussian_ls
│   ├── components
│   │   ├── First_Plasmon_Peak
│   │   ├── Second_Plasmon_

In [11]:
l.generate_param_maps()

In [12]:
l.plot_parameter_maps()

In [9]:
l.Fit_Model.plot(plot_components=True)

In [82]:
l.Fit_Model.red_chisq.plot()

In [58]:
im=l.Fit_Model.
im.plot()
roi=hs.roi.RectangularROI(left=0, right=5., top=0, bottom=5)
imr=roi.interactive(im)
imr.plot()

TypeError: __getitem__() got an unexpected keyword argument 'out'

In [57]:
imr.plot()

In [36]:
roi = hs.roi.RectangularROI(0,0,10,10)
im_roi = roi(a.File)
im_roi.plot()

In [97]:
a.thickness(['Al','Y','Fe'], [0.88,0.07,0.05])
a.thickness_map.plot()
#a.thickness_map.save(filename='thickness map', extension='pgf')
plt.savefig(fname='thickness map', dpi=300, format='pgf')
plt.close()

No information on used beam energy found in metadata! Using default (300 keV).
No information on detector collection angle found in metadata! Using default (10 mrad).


In [64]:
a.File.metadata

├── General
│   ├── original_filename = 18.01.26 Spectrum image EELS.dm3
│   └── title = 18.01.26 Spectrum image EELS
└── Signal
    ├── Noise_properties
    │   └── Variance_linear_model
    │       ├── gain_factor = 1.0
    │       └── gain_offset = 0.0
    ├── binned = False
    ├── quantity = Intensity
    └── signal_type = EELS

In [111]:
import hyperspy.samfire as samf
#model.multifit(fitter='mpfit', method='ls')

#model.store(name='test1')
Smodel = a.Fit_Model.create_samfire(workers=4, ipyparallel=True)
Smodel.strategies

AttributeError: 'NoneType' object has no attribute 'create_samfire'

In [91]:
tolerance = a.Fit_Model.red_chisq.mean(axis=['x','y']).data[0]

In [94]:
Smodel.metadata.goodness_test.tolerance = tolerance*10 # use a sensible value
Smodel.start(fitter='leastsq', method='ls')

In [95]:
print(Smodel.pixels_done)
print(Smodel.pixels_left)

100
0


In [46]:
a.ZLP_Emax.print_summary_statistics()

Summary statistics
------------------
mean:	-0.00483
std:	0.005

min:	-0.0131
Q1:	-0.00917
median:	-0.00487
Q3:	-0.00103
max:	0.00343


In [99]:
Smodel.plot()

In [94]:
a.plot_parameter_maps()

In [60]:
np.max(a.File, axis=a.eels_axis.index_in_axes_manager)

<BaseSignal, title: 18.01.26 Spectrum image EELS, dimensions: (10, 10|)>

In [61]:
a.File/np.max(a.File, axis=a.eels_axis.index_in_axes_manager)

<EELSSpectrum, title: 18.01.26 Spectrum image EELS, dimensions: (10, 10|2048)>

In [96]:
roi = hs.roi.RectangularROI(left=-20., right=-15., top=-165., bottom=-175.)
a.FPP_Emax.metadata.General.title = 'Emax'
a.FPP_Emax.plot()
ac = roi.interactive(a.FPP_Emax)
#ac.plot()

TypeError: '<=' not supported between instances of '_Undefined' and 'float'

In [72]:
signals

[<Signal2D, title: $E_{	extbf{max}}$ - Zero Loss Peak, dimensions: (|20, 20)>,
 <Signal2D, title: $\Gamma$ - Zero Loss Peak, dimensions: (|20, 20)>,
 <Signal2D, title: Emax - Zero Loss Peak, dimensions: (|20, 20)>,
 <Signal2D, title: $\Gamma$ - Zero Loss Peak, dimensions: (|20, 20)>,
 <Signal2D, title: $\Gamma$ - Zero Loss Peak, dimensions: (|20, 20)>,
 <Signal2D, title: $\Gamma$ - Zero Loss Peak, dimensions: (|20, 20)>,
 <Signal2D, title: Intensity - Zero Loss Peak, dimensions: (|20, 20)>,
 <Signal2D, title: $\Gamma$ - Zero Loss Peak, dimensions: (|20, 20)>,
 <Signal2D, title: $Intensity$ - Zero Loss Peak, dimensions: (|20, 20)>,
 None]

In [17]:
a.eels_fit_routine()
a.Fit_Model.plot(plot_components=True)

#a.Fit_Model.enable_background()
#a.Fit_Model.ensure_parameters_in_bounds()

In [8]:
a.Fit_Model.red_chisq.plot()

In [15]:
a.generate_param_maps()

In [16]:
a.Param_Maps[3].plot()

RuntimeError: latex was not able to process the following string:
b'fwhm parameter of First_Plasmon_Peak component Signal'

Here is the full report generated by latex:
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017) (preloaded format=latex)
 restricted \write18 enabled.
entering extended mode
(/home/max/.cache/matplotlib/tex.cache/37df7ff511d07db1b9c75f88652206cd.tex
LaTeX2e <2017-04-15>
Babel <3.9r> and hyphenation patterns for 77 language(s) loaded.
(/usr/share/texlive/texmf-dist/tex/latex/base/article.cls
Document Class: article 2014/09/29 v1.4h Standard LaTeX document class
(/usr/share/texlive/texmf-dist/tex/latex/base/size10.clo))
(/usr/share/texlive/texmf-dist/tex/latex/type1cm/type1cm.sty)
(/usr/share/texlive/texmf-dist/tex/latex/base/textcomp.sty
(/usr/share/texlive/texmf-dist/tex/latex/base/ts1enc.def))
(/usr/share/texlive/texmf-dist/tex/latex/base/inputenc.sty
(/usr/share/texlive/texmf-dist/tex/latex/base/utf8.def
(/usr/share/texlive/texmf-dist/tex/latex/base/t

Traceback (most recent call last):
  File "/home/max/Apps/anaconda3/envs/HS/lib/python3.6/site-packages/matplotlib/texmanager.py", line 298, in _run_checked_subprocess
    stderr=subprocess.STDOUT)
  File "/home/max/Apps/anaconda3/envs/HS/lib/python3.6/subprocess.py", line 336, in check_output
    **kwargs).stdout
  File "/home/max/Apps/anaconda3/envs/HS/lib/python3.6/subprocess.py", line 418, in run
    output=stdout, stderr=stderr)
subprocess.CalledProcessError: Command '['latex', '-interaction=nonstopmode', '--halt-on-error', '/home/max/.cache/matplotlib/tex.cache/37df7ff511d07db1b9c75f88652206cd.tex']' returned non-zero exit status 1.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/max/Apps/anaconda3/envs/HS/lib/python3.6/site-packages/matplotlib/backends/backend_qt5.py", line 519, in _draw_idle
    self.draw()
  File "/home/max/Apps/anaconda3/envs/HS/lib/python3.6/site-packages/matplotlib/backends/backend_agg.p

In [13]:
a.Fit_Model.plot()

In [51]:
np.savetxt('fwhm_first_plasmon',a.Param_Maps[3])

In [116]:
Plasmon=a.Fit_Model.components.First_Plasmon_Peak.

In [119]:
Plasmon_int=Plasmon.integrate1D('Energy loss')

AttributeError: 'Cauchy_Lorentz_Distribution' object has no attribute 'integrate1D'

In [66]:
Plasmon_int.plot()

In [60]:
np.savetxt('Integral_of_First_Plasmon',Plasmon_int)

## Test for Prof. Zaccone/Dr. Rösner --- Mean Dark_STZ

In [162]:
a.Fit_Model.inav[90:97,28:32].components.First_Plasmon_Peak.fwhm.plot()
SB_dark_fwhm=a.Fit_Model.inav[90:97,28:32].components.First_Plasmon_Peak.fwhm.as_signal()

In [163]:
a.Fit_Model.inav[90:97,28:32].components.First_Plasmon_Peak.centre.plot()
SB_dark_centre=a.Fit_Model.inav[90:97,28:32].components.First_Plasmon_Peak.centre.as_signal()

In [164]:
a.Fit_Model.inav[90:97,28:32].components.First_Plasmon_Peak.height.plot()
SB_dark_height=a.Fit_Model.inav[90:97,28:32].components.First_Plasmon_Peak.height.as_signal()

In [165]:
np.savetxt('FürProfZaccone/MeanParameters&Maps/Parameters/dark_fwhm_mean', SB_dark_fwhm.mean((0,1)).data)
np.savetxt('FürProfZaccone/MeanParameters&Maps/Parameters/dark_centre_mean', SB_dark_centre.mean((0,1)).data)
np.savetxt('FürProfZaccone/MeanParameters&Maps/Parameters/dark_height_mean', SB_dark_height.mean((0,1)).data)

In [73]:
im = hs.load('18.01.26 Spectrum image EELS.dm3')
Mean_dark=im.inav[90:97,28:32].sum()/40
model=Mean_dark.create_model(auto_background=False)
function_1 = Cauchy_Lorentz_Distribution(height=60000.,centre=0.,fwhm=0.1)
function_1.name = 'Zero_Loss_Peak'
        
function_2 = Cauchy_Lorentz_Distribution(centre=20.,height=1000.,fwhm=2.5)
function_2.name = 'First_Plasmon_Peak'
        
function_3 = Cauchy_Lorentz_Distribution(centre=30.,height=100.,fwhm=2.)
function_3.name = 'Second_Plasmon_Peak'

model.extend([function_1, function_2, function_3])
model.fit(fitter='leastsq',method='ls')
model.plot(plot_components=True)
model.print_current_values()
model.red_chisq()
np.savetxt('Mean_dark_EELS.txt',Mean_dark)

Components	Parameter	Value
Zero_Loss_Peak
		centre	0.0209587
		fwhm	0.477705
		height	513614
First_Plasmon_Peak
		centre	15.134
		fwhm	1.16522
		height	93803.9
Second_Plasmon_Peak
		centre	30.7834
		fwhm	6.19866
		height	12561.3


## Test for Prof. Zaccone/Dr. Rösner --- Mean Bright_STZ

In [166]:
a.Fit_Model.inav[0:10,30:39].components.First_Plasmon_Peak.fwhm.plot()
SB_bright_fwhm=a.Fit_Model.inav[0:10,30:39].components.First_Plasmon_Peak.fwhm.as_signal()

In [167]:
a.Fit_Model.inav[0:10,30:39].components.First_Plasmon_Peak.centre.plot()
SB_bright_centre=a.Fit_Model.inav[0:10,30:39].components.First_Plasmon_Peak.centre.as_signal()

In [168]:
a.Fit_Model.inav[0:10,30:39].components.First_Plasmon_Peak.height.plot()
SB_bright_height=a.Fit_Model.inav[0:10,30:39].components.First_Plasmon_Peak.height.as_signal()

In [169]:
np.savetxt('FürProfZaccone/MeanParameters&Maps/Parameters/bright_fwhm_mean', SB_bright_fwhm.mean((0,1)).data)
np.savetxt('FürProfZaccone/MeanParameters&Maps/Parameters/bright_centre_mean', SB_bright_centre.mean((0,1)).data)
np.savetxt('FürProfZaccone/MeanParameters&Maps/Parameters/bright_height_mean', SB_bright_height.mean((0,1)).data)

In [72]:
im = hs.load('18.01.26 Spectrum image EELS.dm3')
Mean_bright=im.inav[0:10,30:39].sum()/110
model=Mean_bright.create_model(auto_background=False)
function_1 = Cauchy_Lorentz_Distribution(height=6000000.,centre=0.,fwhm=0.1)
function_1.name = 'Zero_Loss_Peak'
        
function_2 = Cauchy_Lorentz_Distribution(centre=20.,height=100000.,fwhm=2.5)
function_2.name = 'First_Plasmon_Peak'
        
function_3 = Cauchy_Lorentz_Distribution(centre=30.,height=10000.,fwhm=2.)
function_3.name = 'Second_Plasmon_Peak'

model.extend([function_1, function_2, function_3])
model.fit(fitter='leastsq',method='ls')
model.plot(plot_components=True)
model.print_current_values()
model.red_chisq()
np.savetxt('Mean_bright_EELS.txt',Mean_bright)

Components	Parameter	Value
Zero_Loss_Peak
		centre	-0.0198949
		fwhm	-0.480718
		height	597978
First_Plasmon_Peak
		centre	15.1314
		fwhm	-1.25924
		height	96029
Second_Plasmon_Peak
		centre	30.965
		fwhm	6.85571
		height	13207.2


## Test for Prof. Zaccone/Dr. Rösner --- Mean Matrix_near_STZ

In [174]:
a.Fit_Model.inav[80:120,0:20].components.First_Plasmon_Peak.fwhm.plot()
SB_matrix_fwhm=a.Fit_Model.inav[80:120,0:20].components.First_Plasmon_Peak.fwhm.as_signal()

In [175]:
a.Fit_Model.inav[80:120,0:20].components.First_Plasmon_Peak.centre.plot()
SB_matrix_centre=a.Fit_Model.inav[80:120,0:20].components.First_Plasmon_Peak.centre.as_signal()

In [176]:
a.Fit_Model.inav[80:120,0:20].components.First_Plasmon_Peak.height.plot()
SB_matrix_height=a.Fit_Model.inav[80:120,0:20].components.First_Plasmon_Peak.height.as_signal()

In [177]:
np.savetxt('FürProfZaccone/MeanParameters&Maps/Parameters/matrix_fwhm_mean', SB_matrix_fwhm.mean((0,1)).data)
np.savetxt('FürProfZaccone/MeanParameters&Maps/Parameters/matrix_centre_mean', SB_matrix_centre.mean((0,1)).data)
np.savetxt('FürProfZaccone/MeanParameters&Maps/Parameters/matrix_height_mean', SB_matrix_height.mean((0,1)).data)

In [79]:
im = hs.load('18.01.26 Spectrum image EELS.dm3')
Mean_matrix=im.inav[80:120,0:20].sum()/861
model=Mean_matrix.create_model(auto_background=False)
function_1 = Cauchy_Lorentz_Distribution(height=600000.,centre=0.,fwhm=0.1)
function_1.name = 'Zero_Loss_Peak'
        
function_2 = Cauchy_Lorentz_Distribution(centre=20.,height=10000.,fwhm=2.5)
function_2.name = 'First_Plasmon_Peak'
        
function_3 = Cauchy_Lorentz_Distribution(centre=30.,height=1000.,fwhm=2.)
function_3.name = 'Second_Plasmon_Peak'

model.extend([function_1, function_2, function_3])
model.fit(fitter='leastsq',method='ls')
#model.plot(plot_components=True)
#model.print_current_values()
model.red_chisq()
#np.savetxt('Mean_matrix_EELS.txt',Mean_matrix)

array([292144.66689305])

# Testing _Logarithmic-Rescaling_ Script

In [143]:
im = hs.load('Testing/PraktikumTem/cBN_S2_eels_zeroloss_001.dm3')#Postion2 wahrscheinlich mit Carbon+cBN
print(im.metadata)
print(im)

def create_energy_axes(im):
    scale=im.axes_manager[-1].scale
    offset=im.axes_manager[-1].offset
    return np.linspace(0,len(im.data))*scale+offset

def energy_axes_cut(energy):
    calc_index=np.where(energy>=0)+1
    np.copy(energy[calc_ind:-1])

energy=create_energy_axes(im)
im.plot()

def resample_log(energy):
    #tck = interpolate.splrep(x, y, s = sigma)
    #xnew = np.arange(x[0], x[-1], (x[-1]-x[0])/x.size)
    #ynew = interpolate.splev(xnew, tck, der = n)
    x_axes=np.exp(np.arange(np.log(energy[0]), np.log(energy[-1]), np.log(energy[-1]/energy[0])/energy.size))
    #y_axes=
    print(x_axes)
    return x_axes #,y_axes

resample_log(create_energy_axes(im))
#xnew = np.exp(np.arange(np.log(x[0]), np.log(x[-1]), np.log(x[-1]/x[0])/x.size))

├── Acquisition_instrument
│   └── TEM
│       ├── Detector
│       │   └── EELS
│       │       ├── collection_angle = 5.800000190734863
│       │       ├── exposure = 0.004
│       │       └── frame_number = 20
│       ├── acquisition_mode = TEM
│       ├── beam_current = 0.0
│       ├── beam_energy = 200.0
│       ├── convergence_angle = 0.10000000149011612
│       ├── magnification = 80000.0
│       └── microscope = Zeiss
├── General
│   ├── date = 2018-02-16
│   ├── original_filename = cBN_S2_eels_zeroloss_001.dm3
│   ├── time = 14:14:14
│   └── title = cBN_S2_eels_zeroloss_001
└── Signal
    ├── Noise_properties
    │   └── Variance_linear_model
    │       ├── gain_factor = 0.1344086080789566
    │       └── gain_offset = 0.0
    ├── binned = True
    ├── quantity = Electrons (Counts)
    └── signal_type = EELS

<EELSSpectrum, title: cBN_S2_eels_zeroloss_001, dimensions: (|1024)>


C:\Users\m_grov01\AppData\Local\conda\conda\envs\Hyperspy\lib\site-packages\ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in log


ValueError: arange: cannot compute length